# Instructions

* This notebook is refered to MScFE 690 Capstone - Module 4 - Design and it is finished and running succefully, for an ongoing updated code go to the vLive version.
* Since the notebook requires the instalation of modules it is required to restart the sessions afer the modules are installed.


In [127]:
import requests
import pandas as pd
import os
import time
from datetime import date, datetime, timedelta
import json
import re
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.dates as mdates
from scipy.optimize import minimize
from plotly.subplots import make_subplots

from plotly.subplots import make_subplots
import os
import requests
import numpy as np

try:
  import quiverquant
except:
  !pip install quiverquant
  import quiverquant

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
# Create output directory
os.makedirs("data", exist_ok=True)

# Data collection

## Configuration

In [4]:
POLYGON_KEY = "ft2rquZoprD0MSBmJqQfD_8qNsxrkK_1"
QUIVER_TOKEN = "a8dcf66a9fd588b78513ccedb4b92acc948ac6e3"

In [5]:
# Get the directory of this script
SCRIPT_DIR = os.getcwd()
# Congressional term date ranges
CONGRESS_START_DATE = "2019-01-03"  # 116th Congress start
CONGRESS_END_DATE = "2025-01-02"    # 118th Congress end

# Major economic states for geographic filtering
MAJOR_STATES = ['CA', 'NY', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'NC', 'MI',
                'NJ', 'VA', 'WA', 'AZ', 'MA', 'TN', 'CO', 'MD', 'CT', 'AL']

# Output directory
OUTPUT_DIR = os.path.join(SCRIPT_DIR, 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
lawmakers = [
    # --- Senators ---
    # {"name": "Ron Wyden", "party": "Democratic", "chamber": "Senate", "state": "OR", "bioguide": "W000779"},
    # {"name": "Thomas R. Carper", "party": "Democratic", "chamber": "Senate", "state": "DE", "bioguide": "C000174"},
    # {"name": "A. Mitchell Jr. McConnell", "party": "Republican", "chamber": "Senate", "state": "KY", "bioguide": "M000355"},

    # # --- House of Representatives ---

    {"name": "Josh Gottheimer", "party": "Democratic", "chamber": "House", "state": "NJ", "bioguide": "G000583"},
    {"name": "Nancy Pelosi", "party": "Democratic", "chamber": "House", "state": "CA", "bioguide": "P000197"},
    {"name": "Kevin Hern", "party": "Republican", "chamber": "House", "state": "OK", "bioguide": "H001082"},
    {"name": "Mark Green", "party": "Republican", "chamber": "House", "state": "TN", "bioguide": "G000590"},
    {"name": "Bradley S. Schneider", "party": "Democratic", "chamber": "House", "state": "IL", "bioguide": "S001190"},
    {"name": "Virginia Foxx", "party": "Republican", "chamber": "House", "state": "NC", "bioguide": "F000450"},
    {"name": "Kim Schrier", "party": "Democratic", "chamber": "House", "state": "WA", "bioguide": "S001216"},
    {"name": "Mikie Sherrill", "party": "Democratic", "chamber": "House", "state": "NJ", "bioguide": "S001207"},
    {"name": "Daniel Meuser", "party": "Republican", "chamber": "House", "state": "PA", "bioguide": "M001204"},
    {"name": "Susie Lee", "party": "Democratic", "chamber": "House", "state": "NV", "bioguide": "L000590"},
    {"name": "John W. Rose", "party": "Republican", "chamber": "House", "state": "TN", "bioguide": "R000612"},
    {"name": "Brian Mast", "party": "Republican", "chamber": "House", "state": "FL", "bioguide": "M001199"},
    {"name": "Carol Devine Miller", "party": "Republican", "chamber": "House", "state": "WV", "bioguide": "M001205"},
    {"name": "Michael C. Burgess", "party": "Republican", "chamber": "House", "state": "TX", "bioguide": "B001248"},
    {"name": "Pete Sessions", "party": "Republican", "chamber": "House", "state": "TX", "bioguide": "S000250"},
    {"name": "Lois Frankel", "party": "Democratic", "chamber": "House", "state": "FL", "bioguide": "F000462"},
    {"name": "Earl Blumenauer", "party": "Democratic", "chamber": "House", "state": "OR", "bioguide": "B000574"},
    {"name": "Michael Garcia", "party": "Republican", "chamber": "House", "state": "CA", "bioguide": "G000061"},
    {"name": "Donald Sternoff Beyer Jr", "party": "Democratic", "chamber": "House", "state": "VI", "bioguide": "B001292"},
    {"name": "Debbie Dingell", "party": "Democratic", "chamber": "House", "state": "MI", "bioguide": "D000624"},

]
lawmakers = pd.DataFrame(lawmakers)
lawmakers.to_csv("data/lawmakers.csv", index=False)



## Yfinance data

In [8]:
# 1. Fetch Data
ticker = "NVDA"
df = yf.download(ticker, period="1y", interval="1d")
df = df.stack(level='Ticker').reset_index()

# 2. Create Subplots (2 Rows: Price on top, Volume on bottom)
# row_heights=[0.7, 0.3] gives 70% space to Price, 30% to Volume
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=(f'{ticker} Price', 'Volume'),
    row_heights=[0.7, 0.3]
)

# 3. Add Candlestick Trace (Row 1)
fig.add_trace(go.Candlestick(
    x=df.Date,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Price'
), row=1, col=1)

# 4. Add Volume Trace (Row 2)
# We color the volume bars based on price change (Green for up, Red for down)
colors = ['green' if row['Open'] - row['Close'] >= 0
          else 'red' for index, row in df.iterrows()]

fig.add_trace(go.Bar(
    x=df.Date,
    y=df['Volume'],
    marker_color=colors,
    name='Volume'
), row=2, col=1)

# 5. Add Moving Average (Optional, on Row 1)
df['MA20'] = df['Close'].rolling(window=20).mean()
fig.add_trace(go.Scatter(
    x=df.Date,
    y=df['MA20'],
    line=dict(color='orange', width=2),
    name='20-Day MA'
), row=1, col=1)

# 6. Final Layout Adjustments
fig.update_layout(
    title=f'{ticker} Candlestick Chart with Volume',
    template='plotly_dark',
    xaxis_rangeslider_visible=False, # Hides the slider (redundant with zoom)
    height=800 # Taller figure to accommodate both plots
)

# Show the Plot
fig.show()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_26568\1349361877.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\Aaron\AppData\Local\Temp\ipykernel_26568\1349361877.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level='Ticker').reset_index()


In [9]:
df = yf.download(ticker, period="1y", interval="1d", ignore_tz=True)

C:\Users\Aaron\AppData\Local\Temp\ipykernel_26568\462295115.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [10]:
clean_df = df.stack(level='Ticker').reset_index()
clean_df.head()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_26568\1590771923.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Price,Date,Ticker,Close,High,Low,Open,Volume
0,2025-02-07,NVDA,129.804474,130.334328,124.965802,129.184648,228186300
1,2025-02-10,NVDA,133.533478,134.963079,129.924464,130.054419,216989100
2,2025-02-11,NVDA,132.763702,134.443236,130.984190,132.543761,178902400
3,2025-02-12,NVDA,131.104141,132.203847,129.044707,129.984452,160278600
4,2025-02-13,NVDA,135.253006,136.462682,131.134137,131.524030,197430000


## Quiver transaction history

In [19]:
class CustomQuiver(quiverquant.quiver):

    def congress_trading(self, ticker="", politician=True, recent=False, representative="", bioguide=""):


      if recent:
          urlStart = 'https://api.quiverquant.com/beta/live/congresstrading'
      else:
          urlStart = 'https://api.quiverquant.com/beta/bulk/congresstrading'

      if politician:
          representative = representative.replace(" ", "%20")
          ticker = ticker.replace(" ", "%20")

          if representative:
            url = urlStart+"?representative="+representative+ticker

          elif bioguide:
            url = urlStart+"?bioguide_id="+bioguide+ticker


      elif len(ticker)>0:
          url = urlStart+"/"+ticker
      else:
          url = urlStart
      print(url)
      r = requests.get(url, headers=self.headers)
      j = json.loads(r.content)
      df = pd.DataFrame(j)
      return df

# === API KEYS  ===
# - Quiver Quantitative: https://www.quiverquant.com/ (paid subscription required for API access, but has free trial)
# - Documentation on https://api.quiverquant.com/docs/#/ and https://github.com/Quiver-Quantitative/python-api

QUIVER_TOKEN = "a8dcf66a9fd588b78513ccedb4b92acc948ac6e3"

# # 1 Initialize Quiver client (token required for full access)

# qq = quiverquant.quiver(QUIVER_TOKEN)

quiver = CustomQuiver(QUIVER_TOKEN)


In [20]:
def fetch_congress_trading(use_cache=True, cache_file=None):
    """
    Fetch all congress trading data from Quiver API.

    Parameters:
    -----------
    use_cache : bool
        If True, use cached data if available
    cache_file : str
        Path to cache file

    Returns:
    --------
    pd.DataFrame : All congress trading records
    """
    if cache_file is None:
        cache_file = os.path.join(OUTPUT_DIR, 'raw_congress_trades.csv')

    # Check cache
    if use_cache and os.path.exists(cache_file):
        print(f"Loading cached data from {cache_file}")
        df = pd.read_csv(cache_file)
        return df

    # Fetch from API
    print("Fetching congress trading data from Quiver API...")
    quiver = CustomQuiver(QUIVER_TOKEN)
    df = quiver.congress_trading(recent=False)

    # Save cache
    df.to_csv(cache_file, index=False)
    print(f"Saved {len(df)} records to {cache_file}")

    return df

In [21]:
raw_df = fetch_congress_trading(use_cache=True)

Loading cached data from c:\Users\Aaron\Documents\Projects\WQU\Capston_Project\Congress_Trading\output\raw_congress_trades.csv


C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\3193665882.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cache_file)


In [22]:
def filter_by_date_range(df, start_date=CONGRESS_START_DATE, end_date=CONGRESS_END_DATE):
    """
    Filter trading data to specified date range.

    Parameters:
    -----------
    df : pd.DataFrame
        Trading data
    start_date : str
        Start date (YYYY-MM-DD)
    end_date : str
        End date (YYYY-MM-DD)

    Returns:
    --------
    pd.DataFrame : Filtered data
    """
    df_filtered = df.loc[
        (df['Traded'] >= start_date) &
        (df['Traded'] <= end_date)
    ].copy()

    print(f"Filtered to {len(df_filtered)} records between {start_date} and {end_date}")
    return df_filtered


In [23]:
def clean_trading_data(df):
    """
    Clean and preprocess trading data.

    Parameters:
    -----------
    df : pd.DataFrame
        Raw trading data

    Returns:
    --------
    pd.DataFrame : Cleaned data
    """
    df = df.copy()

    # Convert numeric columns
    df['Trade_Size_USD'] = pd.to_numeric(df['Trade_Size_USD'], errors='coerce')
    df['excess_return'] = pd.to_numeric(df['excess_return'], errors='coerce')

    # Drop rows with missing critical values
    df = df.dropna(subset=['Trade_Size_USD', 'excess_return'])

    # Filter out non-positive trade sizes
    df = df[df['Trade_Size_USD'] > 0]

    # Convert dates
    df['Traded'] = pd.to_datetime(df['Traded'])
    df['Filed'] = pd.to_datetime(df['Filed'], errors='coerce')

    # Extract state from District (e.g., 'CA11' -> 'CA')
    df['State'] = df['District'].astype(str).str.strip().str.extract(r'([A-Z]{2})', expand=False)

    # Add year column for yearly analysis
    df['Year'] = df['Traded'].dt.year

    print(f"Cleaned data: {len(df)} records, {df['Name'].nunique()} unique politicians")
    return df



In [24]:
df_date_filtered = filter_by_date_range(raw_df)

Filtered to 56932 records between 2019-01-03 and 2025-01-02


In [25]:
cleaned_df = clean_trading_data(df_date_filtered)

Cleaned data: 52704 records, 244 unique politicians


# EDA

In [82]:
class Congressman:
  def __init__(self, name:str, party:str, chamber:str, state:str, bioguide:str, quiverClient):

    self.name = name
    self.party = party
    self.chamber = chamber
    self.state = state
    self.bioguide = bioguide
# -------------

    self.transactions = self.get_transactions(quiverClient)
    self.tickers = self.get_tickers()

    self.positions = self.get_positions()
    self.positions45days = self.get_positions(delay=True)

    self.portfolio = None
    self.portfolio45days = None

    self.weights = None
    self.weights45days = None

    self.returns = None
    self.returns45days = None

  def get_transactions(self, quiverClient):

    transactions = quiverClient.congress_trading(bioguide=self.bioguide)
    if transactions.empty:
        print("No transacction history for ", self.name)
        return None

    transactions = transactions[~transactions['TickerType'].str.contains('OP|None', na=True)]
    transactions = transactions[['Ticker', 'TickerType', 'Traded', 'Transaction', 'Description', 'Filed', 'Trade_Size_USD']]

    # Removing options trades
    transactions = transactions[~transactions['Description'].str.contains('options', na=False, case=False)]

    transactions['Trade_Size_USD'] = transactions['Trade_Size_USD'].str.extract(r'(\d{1,3}(?:,\d{3})*)', expand=False)

    # Remove commas from the extracted strings and convert to a numeric type (float)
    # Errors='coerce' will turn values that can't be converted into NaN
    transactions['Trade_Size_USD'] = transactions['Trade_Size_USD'].str.replace(',', '', regex=False).astype(float)

    transactions['Transaction'] = transactions['Transaction'].map({"Purchase": 1, "Sale": -1})
    transactions['Transaction'].dropna

    transactions['Trade_Size_USD'] = transactions['Trade_Size_USD'] * transactions['Transaction']

    transactions["Traded"] = pd.to_datetime(transactions["Traded"])
    transactions = transactions.set_index('Traded')

    return transactions

  def get_tickers(self):

    tickers = self.transactions['Ticker'].unique().tolist()

    # Mapping for known ticker changes (keep this)
    ticker_mapping = {'FB': 'META'}  # add more if needed
    tickers = [ticker_mapping.get(t, t) for t in tickers]

    # Strict filter: only allow typical US stock ticker patterns
    # - Uppercase letters, numbers, and dots (for class shares like BRK.B)
    # - No spaces, no commas, no lowercase, reasonable length
    def is_valid_ticker(t):
        return (
            bool(re.fullmatch(r'[A-Z0-9\.]{1,10}', t)) and
            ' ' not in t and
            ',' not in t and
            t.isupper()
        )

    valid_tickers = [t for t in tickers if is_valid_ticker(t)]

    manual_excludes = {'BFET', 'WORK', 'VSNT'}
    valid_tickers = [t for t in valid_tickers if t not in manual_excludes]

    print(valid_tickers)
    return valid_tickers

  def get_positions(self, delay=False):

    if self.transactions.empty:
      # Fallback empty DataFrame
      print('No positions for: ', self.name)
      return pd.DataFrame(columns=self.tickers)

    trans = self.transactions.copy()

    # Apply 45-day delay if requested
    trade_date_col = "Traded"  # default uses original index

    if delay:
        print(f"Delayed for: {self.name}")
        trans['TradedDelay45'] = trans.index + pd.Timedelta(days=45)
        trade_date_col = "TradedDelay45"

    # Daily net dollar flows by ticker
    daily_delta = trans.pivot_table(
        index=trade_date_col,
        columns="Ticker",
        values="Trade_Size_USD",
        aggfunc="sum"
    ).sort_index()

    print(f"Daily delta for {self.name}:")
    positions = daily_delta.cumsum()

    full_index = pd.date_range(
        start=positions.index.min(),
        end=positions.index.max(),
        freq="D"
    )

    # Reindex to daily, forward-fill holdings, zero before first trade
    positions = (
        positions
        .reindex(full_index)
        .ffill()
        .fillna(0)
    )

    start_date_str = '2019-01-03'
    end_date_str = '2025-01-03'
    start = date.fromisoformat(start_date_str)
    end = date.fromisoformat(end_date_str)

    # Final safety: only our tickers, sorted columns
    target_index = pd.date_range(start=start, end=end, freq='D')
    positions = positions.reindex(target_index)

    positions = positions.loc[:, positions.columns.isin(self.tickers)]

    return positions

# -----

lawmakers_test = [
    # --- Senators ---
    {"name": "Mitch McConnell", "party": "Republican", "chamber": "Senate", "state": "KY", "bioguide": "M000355"},
    {"name": "Nancy Pelosi", "party": "Democrat", "chamber": "House", "state": "CA", "bioguide": "P000197"},
    {"name": "Ted Cruz", "party": "Republican", "chamber": "Senate", "state": "TX", "bioguide": "C001098"},  # state fixed from "ME"
  ]

congressmen = []

for l in lawmakers_test:
    congressman = Congressman(
        name=l["name"],
        party=l["party"],
        chamber=l["chamber"],
        state=l["state"],
        bioguide=l["bioguide"],
        quiverClient=quiver
    )
    congressmen.append(congressman)



all_tickers = [ticker for c in congressmen for ticker in c.tickers]
unique_tickers = list(set(all_tickers))  # removes duplicates
unique_tickers.sort()                    # optional: alphabetical order


def get_prices(tickers):
  start_date_str = '2019-01-03'
  end_date_str = '2025-01-03'
  start_date = date.fromisoformat(start_date_str)
  end_date = date.fromisoformat(end_date_str)

  all_ticker_prices = {}

  if tickers:
      print(f"Downloading prices for {len(tickers)} tickers...")
      prices_multi = yf.download(
          tickers,
          start=start_date_str,
          end=end_date_str,
          interval='1d',
          auto_adjust=True,
          progress=True
      )

      # prices_multi['Close'] is a DataFrame with columns = tickers
      close_df = prices_multi['Close']

      for ticker in tickers:
          series = close_df[ticker]
          if series.notna().any():  # has at least some real data
              all_ticker_prices[ticker] = series
          else:
              print(f"No valid price data for {ticker}")

  else:
      print("No valid tickers found.")

  # Create a master DataFrame with a full date range
  full_date_range = pd.date_range(start=start_date_str, end=end_date_str, freq='D')
  prices_df = pd.DataFrame(index=full_date_range)

  # Populate the master DataFrame with ticker prices
  for ticker, prices_series in all_ticker_prices.items():
      prices_df[ticker] = prices_series

  prices_df = prices_df.dropna(how='all')
  prices_df = prices_df.fillna(0)

  return prices_df

prices = get_prices(unique_tickers)
print(prices)


https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=M000355
['LAZR', 'WFC', 'KR', 'IR', 'VMC']
Daily delta for Mitch McConnell:
Delayed for: Mitch McConnell
Daily delta for Mitch McConnell:
https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=P000197
['AB', 'AAPL', 'GOOGL', 'PYPL', 'DIS', 'NVDA', 'AMZN', 'MSFT', 'V', 'TSLA', 'NFLX', 'RBLX', 'WBD', 'CRWD', 'META', 'MORN', 'IBKR', 'T']
Daily delta for Nancy Pelosi:
Delayed for: Nancy Pelosi
Daily delta for Nancy Pelosi:
https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=C001098


[                       0%                       ]

['GS', 'BTC', 'PAGP', 'OKE']
Daily delta for Ted Cruz:
Delayed for: Ted Cruz
Daily delta for Ted Cruz:


[*********************100%***********************]  27 of 27 completed

                  AAPL         AB        AMZN        BTC        CRWD  \
2019-01-03   33.799667  15.191701   75.014000   0.000000    0.000000   
2019-01-04   35.242558  15.540108   78.769501   0.000000    0.000000   
2019-01-07   35.164112  15.462683   81.475502   0.000000    0.000000   
2019-01-08   35.834442  15.540108   82.829002   0.000000    0.000000   
2019-01-09   36.442986  15.517994   82.971001   0.000000    0.000000   
...                ...        ...         ...        ...         ...   
2024-12-26  257.853760  34.239243  227.050003  42.330002  365.079987   
2024-12-27  254.439224  34.083652  223.750000  41.820000  354.989990   
2024-12-30  251.064484  33.369759  221.300003  41.750000  349.309998   
2024-12-31  249.292511  33.946365  219.389999  41.869999  342.160004   
2025-01-02  242.752090  33.918907  220.220001  43.160000  347.339996   

                   DIS       GOOGL          GS       IBKR         IR  ...  \
2019-01-03  102.629349   50.888996  144.015839  12.552111 

In [ ]:
congressmen[0].positions

In [ ]:
congressmen[1].transactions

In [ ]:
congressmen[2].transactions

In [ ]:
congressmen[2].name

In [ ]:
nancy = quiver.congress_trading(bioguide="P000197")

In [89]:
john = Congressman(name="John W. Rose", party="Republican", chamber="House", state="TN", bioguide="R000612", quiverClient=quiver)

https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=R000612
['FNSXX', 'GOOG', 'SMCWX', 'SYMBOL', 'MSFT', 'GOOGL', 'CLR', 'WFC', 'BAC', 'PNFP', 'CLB', 'CAT']
Daily delta for John W. Rose:
Delayed for: John W. Rose
Daily delta for John W. Rose:


In [90]:
john.transactions

,Ticker,TickerType,Transaction,Description,Filed,Trade_Size_USD
Traded,,,,,,
2025-06-23,FNSXX,OT,-1,FIMM MONEY MARKET PORT FUND INSTL CL M/M (FNSXX),2025-07-04,-150.0
2025-06-23,SYMBOL: OGVXX,OT,-1,JPMORGAN U.S. GOVERNMENT MONEY MARKET FD CAPIT...,2025-07-04,-100.0
2025-06-03,SYMBOL: AEPGX,OT,-1,AMERICAN FUNDS EUPAC FUND CL A M/F (SYMBOL: AE...,2025-07-04,-250.0
2025-06-03,GOOG,ST,-1,None,2025-07-04,-250.0
2025-06-03,SMCWX,OT,-1,AMERICAN FUNDS SMALLCAP WORLD FUND CL A M/F (S...,2025-07-04,-500.0
2025-06-03,SYMBOL,OT,-1,AMERICAN FUNDS SMALLCAP WORLD FUND CL A M/F (S...,2025-07-04,-500.0
2025-06-03,SYMBOL: CWGIX,OT,-1,AMERICAN FUNDS CAPITAL WORLD GROWTH & INCOME F...,2025-07-04,-250.0
2025-06-03,MSFT,ST,-1,None,2025-07-04,-250.0
2025-06-03,SYMBOL: OGVXX,OT,1,JPMORGAN U.S. GOVERNMENT MONEY MARKET FD CAPIT...,2025-07-04,100.0


In [91]:
john = quiver.congress_trading(bioguide="R000612")

https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=R000612


In [92]:
john[john.TickerType=='ST']

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
3,GOOG,ST,None,2025-06-03,Sale,250001.0,None,None,None,John W. Rose,R000612,2025-07-04,R,TN06,Representatives,None,None,76.7342780181465,None,2025-07-07
7,MSFT,ST,None,2025-06-03,Sale,250001.0,None,None,None,John W. Rose,R000612,2025-07-04,R,TN06,Representatives,None,None,-29.2747850320266,None,2025-07-07
11,GOOGL,ST,None,2025-06-03,Sale,1000001.0,None,None,None,John W. Rose,R000612,2025-07-04,R,TN06,Representatives,None,None,78.3635992279939,None,2025-07-07
12,CLR,ST,None,2022-11-29,Sale,15001.0,None,None,THIS TRANSACTION WAS A FORCED SALE AS THE RESU...,John W. Rose,R000612,2022-12-30,R,TN06,Representatives,None,None,None,None,2023-11-16
13,WFC,ST,None,2019-11-12,Sale,100001.0,None,None,None,John W. Rose,R000612,2019-12-11,R,TN06,Representatives,None,None,-50.5133681126724,None,2023-11-16
14,GOOGL,ST,None,2019-11-12,Sale,100001.0,None,None,None,John W. Rose,R000612,2019-12-11,R,TN06,Representatives,None,None,273.950197226408,None,2023-11-16
15,BAC,ST,None,2019-11-12,Sale,250001.0,None,None,None,John W. Rose,R000612,2019-12-11,R,TN06,Representatives,None,None,-52.9886880403614,None,2023-11-16
16,PNFP,ST,None,2019-11-12,Sale,100001.0,None,None,None,John W. Rose,R000612,2019-12-11,R,TN06,Representatives,None,None,-52.8510455642803,None,2023-11-16
17,CLB,ST,None,2019-11-12,Sale,1001.0,None,None,None,John W. Rose,R000612,2019-12-11,R,TN06,Representatives,None,None,-184.226908401677,None,2023-11-16
18,MSFT,ST,None,2019-11-12,Sale,500001.0,None,None,None,John W. Rose,R000612,2019-12-11,R,TN06,Representatives,None,None,48.9286717102281,None,2023-11-16


In [93]:
bradley = quiver.congress_trading(bioguide="S001190")

https://api.quiverquant.com/beta/bulk/congresstrading?bioguide_id=S001190


In [95]:
bradley[(bradley.TickerType=='ST') & (bradley.Transaction=="Purchase")]

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
31,PGR,ST,None,2019-05-24,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-06-21,D,IL10,Representatives,None,None,7.55430220250236,None,2023-11-16
32,FISV,ST,None,2019-05-24,Purchase,50001.0,None,None,None,Bradley S. Schneider,S001190,2019-06-21,D,IL10,Representatives,None,None,-175.569853779365,None,2023-11-16
34,TDOC,ST,None,2019-05-07,Purchase,50001.0,None,None,None,Bradley S. Schneider,S001190,2019-06-21,D,IL10,Representatives,None,None,-231.940136905252,None,2023-11-16
37,FISV,ST,None,2019-05-03,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-06-17,D,IL10,Representatives,None,None,-166.058273993189,None,2023-11-16
39,QCOM,ST,None,2019-04-24,Purchase,250001.0,None,None,None,Bradley S. Schneider,S001190,2019-05-26,D,IL10,Representatives,None,None,-78.531920202045,None,2023-11-16
41,PGR,ST,None,2019-03-11,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-04-06,D,IL10,Representatives,None,None,29.5589241939478,None,2023-11-16
43,MIDD,ST,None,2019-03-05,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-04-06,D,IL10,Representatives,None,None,-117.732784337858,None,2023-11-16
45,CDW,ST,None,2019-02-19,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-04-06,D,IL10,Representatives,None,None,-98.0703816542529,None,2023-11-16
46,BABA,ST,None,2019-02-19,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-04-06,D,IL10,Representatives,None,None,-153.211087811798,None,2023-11-16
48,CDW,ST,None,2019-02-12,Purchase,100001.0,None,None,None,Bradley S. Schneider,S001190,2019-02-17,D,IL10,Representatives,None,None,-98.2614185737568,None,2023-11-16


# Analysis

In [26]:
INITIAL_CAPITAL = 100000  # For normalization

# Data file paths
RAW_TRADES_PATH = './output/raw_congress_trades.csv'
LAWMAKERS_PATH = './data/lawmakers.csv'

# Snapshot dates for pie charts
SNAPSHOT_DATES = [
    '2019-01-03',  # Start
    '2019-12-31',  # Year-end 2019
    '2020-12-31',  # Year-end 2020
    '2021-12-31',  # Year-end 2021
    '2022-12-31',  # Year-end 2022
    '2023-12-31',  # Year-end 2023
    '2024-12-31',  # Year-end 2024
    '2025-01-02',  # End
]


In [27]:
cleaned_df.columns

Index(['Ticker', 'TickerType', 'Company', 'Traded', 'Transaction',
       'Trade_Size_USD', 'Status', 'Subholding', 'Description', 'Name',
       'BioGuideID', 'Filed', 'Party', 'District', 'Chamber', 'Comments',
       'Quiver_Upload_Time', 'excess_return', 'State', 'last_modified',
       'Year'],
      dtype='object')

In [28]:
class Congressman_df:
    """Represents a legislator with their trading history and positions.

    Uses pre-loaded DataFrame instead of API calls.
    """

    def __init__(self, name: str, party: str, chamber: str, state: str,
                 bioguide: str, trades_df: pd.DataFrame):
        self.name = name
        self.party = party
        self.chamber = chamber
        self.state = state
        self.bioguide = bioguide

        self.transactions = self._get_transactions(trades_df)
        self.tickers = self._get_tickers() if self.transactions is not None else []
        self.positions_df = self._build_positions_df() if self.transactions is not None else pd.DataFrame()

    def _get_transactions(self, trades_df: pd.DataFrame):
        """Filter and clean transaction history from DataFrame."""
        try:
            # Filter by BioGuideID
            transactions = trades_df[trades_df['BioGuideID'] == self.bioguide].copy()

            if transactions.empty:
                # Try by name as fallback
                transactions = trades_df[trades_df['Name'] == self.name].copy()

            if transactions.empty:
                print(f"No transactions found for {self.name}")
                return None

            # Filter out options and non-stock trades
            if 'TickerType' in transactions.columns:
                transactions = transactions[~transactions['TickerType'].str.contains('OP|None', na=True)]

            # Keep relevant columns
            keep_cols = ['Ticker', 'TickerType', 'Traded', 'Transaction',
                         'Description', 'Filed', 'Trade_Size_USD']
            available_cols = [c for c in keep_cols if c in transactions.columns]
            transactions = transactions[available_cols].copy()

            # Filter out options from description
            if 'Description' in transactions.columns:
                transactions = transactions[~transactions['Description'].str.contains('options', na=False, case=False)]

            # Parse trade size - handle both string and numeric values
            transactions['Trade_Size_USD'] = pd.to_numeric(transactions['Trade_Size_USD'], errors='coerce')

            # Convert dates
            transactions['Traded'] = pd.to_datetime(transactions['Traded'])
            transactions['Filed'] = pd.to_datetime(transactions['Filed'])

            # Drop rows with missing trade size
            # transactions = transactions.dropna(subset=['Trade_Size_USD'])

            return transactions.sort_values('Traded').reset_index(drop=True)

        except Exception as e:
            print(f"Error processing transactions for {self.name}: {e}")
            return None

    def _get_tickers(self):
        """Extract unique valid stock tickers."""
        if self.transactions is None:
            return []

        tickers = self.transactions['Ticker'].unique().tolist()

        # Mapping for known ticker changes
        ticker_mapping = {'FB': 'META'}
        tickers = [ticker_mapping.get(t, t) for t in tickers]

        # Validate ticker format
        def is_valid_ticker(t):
            return (
                bool(re.fullmatch(r'[A-Z0-9\.]{1,10}', str(t))) and
                ' ' not in str(t) and
                ',' not in str(t) and
                str(t).isupper()
            )

        valid_tickers = [t for t in tickers if is_valid_ticker(t)]
        return valid_tickers

    def _build_positions_df(self):
        """Build cumulative positions DataFrame."""
        if self.transactions is None or self.transactions.empty:
            return pd.DataFrame()

        records = []
        for _, tx in self.transactions.iterrows():
            ticker = tx['Ticker']
            traded_date = tx['Traded']
            amount = tx['Trade_Size_USD']

            # Determine direction
            if tx['Transaction'] in ['Purchase', 'Buy']:
                sign = 1
            elif tx['Transaction'] in ['Sale', 'Sell', 'Sale (Full)', 'Sale (Partial)']:
                sign = -1
            else:
                continue

            records.append({
                'Date': traded_date,
                'Ticker': ticker,
                'Amount': sign * amount,
                'Filed': tx['Filed'],
                'Trade_Size_USD': amount
            })

        if not records:
            return pd.DataFrame()

        return pd.DataFrame(records).sort_values('Date').reset_index(drop=True)

    def get_position(self, as_of_date):
        """Get portfolio positions as of a specific date (in dollar amounts)."""
        if self.positions_df.empty:
            return {}

        as_of = pd.to_datetime(as_of_date)
        df = self.positions_df[self.positions_df['Date'] <= as_of]

        if df.empty:
            return {}

        positions = df.groupby('Ticker')['Amount'].sum().to_dict()
        # Only keep positive positions
        return {k: v for k, v in positions.items() if v > 0}


    def get_portfolio_composition(self, as_of_date, prices_df):
        """Get detailed portfolio composition with current market values.

        Args:
            as_of_date: Date to get composition for
            prices_df: DataFrame of prices from fetch_prices()

        Returns:
            DataFrame with columns: Ticker, Shares, Price, Value, Weight
        """
        as_of = pd.to_datetime(as_of_date)

        # Get cumulative positions in dollars
        dollar_positions = self.get_position(as_of)

        if not dollar_positions:
            print(f"No positions found for {self.name} as of {as_of_date}")
            return pd.DataFrame()

        records = []
        for ticker, dollar_amount in dollar_positions.items():
            # Get price on the as_of date
            price = get_price_on_date(prices_df, ticker, as_of)

            if price and price > 0:
                shares = dollar_amount / price
                current_value = shares * price
                records.append({
                    'Ticker': ticker,
                    'Cost_Basis_USD': dollar_amount,
                    'Shares': shares,
                    'Price': price,
                    'Value': current_value
                })

        if not records:
            return pd.DataFrame()

        df = pd.DataFrame(records)
        total_value = df['Value'].sum()
        df['Weight'] = (df['Value'] / total_value * 100).round(2)
        df = df.sort_values('Value', ascending=False).reset_index(drop=True)

        # Add summary info
        print(f"\n{'='*60}")
        print(f"Portfolio Composition for {self.name}")
        print(f"As of: {as_of_date}")
        print(f"{'='*60}")
        print(f"Total Value: ${total_value:,.2f}")
        print(f"Number of Positions: {len(df)}")
        print(f"{'='*60}\n")

        return df




In [29]:
def fetch_prices(tickers: list, start_date=CONGRESS_START_DATE, end_date=CONGRESS_END_DATE):
    """Fetch historical prices for all tickers."""
    print(f"Fetching prices for {len(tickers)} tickers...")

    # Add buffer for date lookups
    start_dt = pd.to_datetime(start_date) - timedelta(days=30)
    end_dt = pd.to_datetime(end_date) + timedelta(days=5)

    all_prices = {}
    failed_tickers = []

    for i, ticker in enumerate(tickers):
        try:
            data = yf.download(ticker, start=start_dt, end=end_dt, progress=False)
            if not data.empty:
                # Handle potential multi-level columns from yfinance
                close_col = data['Close']
                if isinstance(close_col, pd.DataFrame):
                    # Multi-level columns - flatten
                    close_col = close_col.iloc[:, 0]
                # Ensure we have a Series with float values
                all_prices[ticker] = close_col.astype(float)
            else:
                failed_tickers.append(ticker)
        except Exception as e:
            failed_tickers.append(ticker)

        if (i + 1) % 50 == 0:
            print(f"  Fetched {i+1}/{len(tickers)} tickers")
            time.sleep(1)

    if failed_tickers:
        print(f"Failed to fetch: {len(failed_tickers)} tickers")

    print(f"Successfully fetched prices for {len(all_prices)} tickers")

    prices_df = pd.DataFrame(all_prices)
    prices_df.index = pd.to_datetime(prices_df.index)

    return prices_df


In [30]:
def get_price_on_date(prices_df, ticker, date, lookback_days=5):
    """Get close price for a ticker on a date (with lookback for non-trading days)."""
    date = pd.to_datetime(date)

    if ticker not in prices_df.columns:
        return None

    ticker_prices = prices_df[ticker].dropna()
    # Try exact date first
    if date in ticker_prices.index:
        return ticker_prices.loc[date]

    # Look back up to lookback_days
    for i in range(1, lookback_days + 1):
        check_date = date - timedelta(days=i)
        if check_date in ticker_prices.index:
            return ticker_prices.loc[check_date]
    return None

## Copy-cat portfolio

In [102]:
class CopycatPortfolio:
    """Simulates a copy-cat portfolio following a legislator's trades.

    Key Logic:
    - Start with INITIAL_CAPITAL as cash
    - Initialize with existing positions from get_position(start_date)
    - Execute trades on Filed date using Filed date prices
    - Shares calculated as: Trade_Size_USD / Traded_date_price
    - Daily portfolio value = cash + sum(shares * current_price)
    """

    def __init__(self, congressman, prices_df):
        self.congressman = congressman
        self.prices_df = prices_df
        self.holdings = {}  # {ticker: shares}
        self.cash = INITIAL_CAPITAL
        self.history = []
        self.snapshots = {}  # {date_str: holdings_dict}

    def simulate(self, start_date=CONGRESS_START_DATE, end_date=CONGRESS_END_DATE):
        """Run the day-by-day simulation."""
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)

        # Reset state
        self.holdings = {}
        self.cash = INITIAL_CAPITAL

        # Initialize with starting positions from before simulation start
        initial_positions = self.congressman.get_position(start_dt)
        if initial_positions:
            print(f"    Initial positions: {len(initial_positions)} tickers")
            for ticker, dollar_amount in initial_positions.items():
                price = get_price_on_date(self.prices_df, ticker, start_dt)
                if price is not None and price > 0:
                    shares = dollar_amount / price
                    self.holdings[ticker] = shares
                    # Don't reduce cash - we're copying existing positions, not buying them

        # Check if there are any transactions
        if self.congressman.positions_df.empty:
            print(f"    No new transactions to simulate")
            # Still generate returns based on initial holdings
            return self._generate_daily_values(start_dt, end_dt)

        # Get transactions within simulation period (by Filed date)
        txs = self.congressman.positions_df.copy()
        txs['Filed'] = pd.to_datetime(txs['Filed'])
        txs = txs[(txs['Filed'] >= start_dt) & (txs['Filed'] <= end_dt)]

        if txs.empty:
            print(f"    No transactions in date range")
            return self._generate_daily_values(start_dt, end_dt)

        print(f"    Transactions in range: {len(txs)}")

        # Prepare transactions grouped by Filed date
        # Adjust weekend Filed dates to next business day (Monday)
        def adjust_to_next_business_day(d):
            d = pd.to_datetime(d)
            if d.weekday() == 5:  # Saturday -> Monday
                return (d + timedelta(days=2)).date()
            elif d.weekday() == 6:  # Sunday -> Monday
                return (d + timedelta(days=1)).date()
            return d.date()
        
        txs['Filed_date'] = txs['Filed'].apply(adjust_to_next_business_day)
        trades_by_date = txs.groupby('Filed_date')

        # Generate daily values with trade execution
        date_range = pd.date_range(start=start_dt, end=end_dt, freq='B')  # Business days

        records = []
        initial_stock_value = None
        prev_stock_value = None

        for date in date_range:
            date_key = date.date()

            # Check if there are trades to execute on this date
            if date_key in trades_by_date.groups:
                day_trades = trades_by_date.get_group(date_key)
                self._execute_trades(day_trades, date)

            # Calculate stock holdings value
            stock_value = self._calculate_stock_holdings_value(date)

            # Cash invested = INITIAL_CAPITAL - current_cash
            cash_invested = INITIAL_CAPITAL - self.cash

            # Track initial stock value for cumulative return
            if initial_stock_value is None:
                initial_stock_value = stock_value

            # Cumulative return = (stock_value - initial_stock_value) / cash_invested
            if cash_invested > 0:
                cumulative_return = (stock_value - initial_stock_value) / cash_invested
            else:
                cumulative_return = 0

            # Log return (for plotting)
            if prev_stock_value is not None and prev_stock_value > 0 and stock_value > 0:
                log_return = np.log(stock_value / prev_stock_value)
            else:
                log_return = 0

            records.append({
                'Date': date,
                'stock_value': stock_value,
                'cash': self.cash,
                'cash_invested': cash_invested,
                'portfolio_value': stock_value + self.cash,
                'cumulative_return': cumulative_return,
                'cumulative_return_pct': cumulative_return * 100,
                'log_return': log_return
            })

            prev_stock_value = stock_value

            # Capture snapshots
            date_str = date.strftime('%Y-%m-%d')
            if date_str in SNAPSHOT_DATES:
                self.snapshots[date_str] = self._get_current_holdings_in_dollars(date)

        df = pd.DataFrame(records)

        if not df.empty:
            # Cumulative log return (sum of daily log returns)
            df['cumulative_log_return'] = df['log_return'].cumsum()
        else:
            print(f"    WARNING: No records generated!")

        self.history = df
        return df

    def _generate_daily_values(self, start_dt, end_dt):
        """Generate daily portfolio values for portfolios with no new trades (uses initial holdings)."""
        date_range = pd.date_range(start=start_dt, end=end_dt, freq='B')

        records = []
        initial_stock_value = None
        prev_stock_value = None

        for date in date_range:
            # Calculate stock holdings value
            stock_value = self._calculate_stock_holdings_value(date)
            cash_invested = INITIAL_CAPITAL - self.cash

            # Track initial stock value
            if initial_stock_value is None:
                initial_stock_value = stock_value

            # Cumulative return = (stock_value - initial_stock_value) / cash_invested
            if cash_invested > 0:
                cumulative_return = (stock_value - initial_stock_value) / cash_invested
            else:
                cumulative_return = 0

            # Log return (for plotting)
            if prev_stock_value is not None and prev_stock_value > 0 and stock_value > 0:
                log_return = np.log(stock_value / prev_stock_value)
            else:
                log_return = 0

            records.append({
                'Date': date,
                'stock_value': stock_value,
                'cash': self.cash,
                'cash_invested': cash_invested,
                'portfolio_value': stock_value + self.cash,
                'cumulative_return': cumulative_return,
                'cumulative_return_pct': cumulative_return * 100,
                'log_return': log_return
            })

            prev_stock_value = stock_value

            # Capture snapshots
            date_str = date.strftime('%Y-%m-%d')
            if date_str in SNAPSHOT_DATES:
                self.snapshots[date_str] = self._get_current_holdings_in_dollars(date)

        df = pd.DataFrame(records)

        if not df.empty:
            # Cumulative log return (sum of daily log returns)
            df['cumulative_log_return'] = df['log_return'].cumsum()

        self.history = df
        return df

    def _execute_trades(self, day_trades, current_date):
        """Execute all trades for a given day."""
        for _, tx in day_trades.iterrows():
            ticker = tx['Ticker']
            traded_date = tx['Date']  # Traded date
            trade_size_usd = tx['Trade_Size_USD']
            is_buy = tx['Amount'] > 0

            # Calculate shares using Traded date price
            traded_price = get_price_on_date(self.prices_df, ticker, traded_date)

            # Execute at current (Filed) date price
            filed_price = get_price_on_date(self.prices_df, ticker, current_date)

            if traded_price is None or filed_price is None or traded_price <= 0 or filed_price <= 0:
                continue

            # Shares = Trade_Size_USD / Traded_date_price
            shares = trade_size_usd / traded_price

            if is_buy:
                # Buy shares at Filed price
                cost = shares * filed_price
                self.holdings[ticker] = self.holdings.get(ticker, 0) + shares
                self.cash -= cost
            else:
                # Sell shares at Filed price
                if ticker in self.holdings:
                    shares_to_sell = min(shares, self.holdings[ticker])
                    self.holdings[ticker] -= shares_to_sell
                    self.cash += shares_to_sell * filed_price

                    if self.holdings[ticker] <= 0.0001:
                        del self.holdings[ticker]

    def _calculate_portfolio_value(self, date):
        """Calculate total portfolio value on a given date."""
        portfolio_value = self.cash

        for ticker, shares in self.holdings.items():
            price = get_price_on_date(self.prices_df, ticker, date)
            if price is not None and price > 0:
                portfolio_value += shares * price

        return portfolio_value

    def _calculate_stock_holdings_value(self, date):
        """Calculate total value of stock holdings only (excluding cash)."""
        stock_value = 0

        for ticker, shares in self.holdings.items():
            price = get_price_on_date(self.prices_df, ticker, date)
            if price is not None and price > 0:
                stock_value += shares * price

        return stock_value

    def _get_current_holdings_in_dollars(self, date):
        """Get current holdings valued in dollars."""
        holdings = {}
        for ticker, shares in self.holdings.items():
            price = get_price_on_date(self.prices_df, ticker, date)
            if price and price > 0:
                holdings[ticker] = shares * price
        return holdings

    def get_portfolio_breakdown(self, as_of_date=None):
        """Get detailed portfolio breakdown including cash at a specific date.

        Args:
            as_of_date: Optional date to check. If None, uses current state.
                        Note: For historical dates, you must run simulate() first.

        Returns:
            DataFrame with all positions including cash, with columns:
            Asset, Shares, Price, Value, Weight
        """
        if as_of_date:
            as_of = pd.to_datetime(as_of_date)
        else:
            as_of = None

        records = []
        total_stock_value = 0

        # Add stock holdings
        for ticker, shares in self.holdings.items():
            if as_of:
                price = get_price_on_date(self.prices_df, ticker, as_of)
            else:
                # Use most recent price
                if ticker in self.prices_df.columns:
                    price = self.prices_df[ticker].dropna().iloc[-1]
                else:
                    price = None

            if price and price > 0:
                value = shares * price
                total_stock_value += value
                records.append({
                    'Asset': ticker,
                    'Type': 'Stock',
                    'Shares': shares,
                    'Price': price,
                    'Value': value
                })

        # Add cash
        records.append({
            'Asset': 'CASH',
            'Type': 'Cash',
            'Shares': None,
            'Price': 1.0,
            'Value': self.cash
        })

        df = pd.DataFrame(records)
        total_value = df['Value'].sum()
        df['Weight'] = (df['Value'] / total_value * 100).round(2)
        df = df.sort_values('Value', ascending=False).reset_index(drop=True)

        # Print summary
        print(f"\n{'='*70}")
        print(f"Portfolio Breakdown for {self.congressman.name}")
        if as_of_date:
            print(f"As of: {as_of_date}")
        print(f"{'='*70}")
        print(f"Total Portfolio Value: ${total_value:,.2f}")
        print(f"  - Stock Holdings:    ${total_stock_value:,.2f} ({total_stock_value/total_value*100:.1f}%)")
        print(f"  - Cash:              ${self.cash:,.2f} ({self.cash/total_value*100:.1f}%)")
        print(f"Number of Positions:   {len(self.holdings)}")
        print(f"{'='*70}\n")

        return df


In [103]:
def get_spy_returns(start_date=CONGRESS_START_DATE, end_date=CONGRESS_END_DATE):
    """Get SPY benchmark returns."""
    spy = yf.download('SPY', start=start_date, end=end_date, progress=False)

    df = pd.DataFrame({
        'Date': spy.index,
        'Close': spy['Close'].values.flatten()
    })

    df['daily_return'] = df['Close'].pct_change()
    df['cumulative_return'] = (1 + df['daily_return'].fillna(0)).cumprod() - 1
    df['cumulative_return_pct'] = df['cumulative_return'] * 100

    # Log returns
    df['log_return'] = np.log(1 + df['daily_return']).replace([np.inf, -np.inf], 0).fillna(0)
    df['cumulative_log_return'] = df['log_return'].cumsum()

    return df


In [104]:
congressmen = []
all_tickers = set()

for _, row in lawmakers.iterrows():
    congressman = Congressman_df(
        name=row['name'],
        party=row['party'],
        chamber=row['chamber'],
        state=row['state'],
        bioguide=row['bioguide'],
        trades_df=raw_df
    )
    congressmen.append(congressman)
    all_tickers.update(congressman.tickers)


In [105]:
raw_df[raw_df.Name=="Nancy Pelosi"]

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,BioGuideID,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
28,AB,AB,NaN,2026-01-16,Purchase,1000001.0,NaN,NaN,"PURCHASED 25,000 SHARES.",Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,NaN,NaN,3.231316,NaN,2026-01-26
37,GOOGL,ST,NaN,2026-01-16,Purchase,500001.0,NaN,NaN,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,NaN,NaN,3.276626,NaN,2026-01-26
40,NVDA,ST,NaN,2026-01-16,Purchase,250001.0,NaN,NaN,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,NaN,NaN,-3.694373,NaN,2026-01-26
44,AMZN,ST,NaN,2026-01-16,Purchase,500001.0,NaN,NaN,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,NaN,NaN,0.182836,NaN,2026-01-26
52,VST,ST,NaN,2026-01-16,Purchase,100001.0,NaN,NaN,EXERCISED 50 CALL OPTIONS PURCHASED 1/14/25 (5...,Nancy Pelosi,P000197,2026-01-23,D,CA11,Representatives,NaN,NaN,-8.085963,NaN,2026-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103919,DIS,NaN,NaN,2014-11-28,Purchase,1001.0,NaN,NaN,PURCHASE OF 8 OPTIONS,Nancy Pelosi,P000197,2014-12-17,D,CA12,Representatives,NaN,NaN,-224.964767,NaN,2023-11-16
103928,HTZ,NaN,NaN,2014-11-28,Purchase,50001.0,NaN,NaN,PURCHASE OF 100 OPTIONS,Nancy Pelosi,P000197,2014-12-17,D,CA12,Representatives,NaN,NaN,-315.164299,NaN,2023-11-16
103940,HTZ,NaN,NaN,2014-11-25,Purchase,100001.0,NaN,NaN,PURCHASE OF 200 OPTIONS,Nancy Pelosi,P000197,2014-12-17,D,CA12,Representatives,NaN,NaN,-316.483220,NaN,2023-11-16
104113,HTZ,NaN,NaN,2014-11-06,Purchase,50001.0,NaN,NaN,PURCHASE OF 150 OPTIONS,Nancy Pelosi,P000197,2014-12-17,D,CA12,Representatives,NaN,NaN,-320.077490,NaN,2023-11-16


In [106]:
congressmen[9].name

'Susie Lee'

In [107]:
congressmen[9].transactions

,Ticker,TickerType,Traded,Transaction,Description,Filed,Trade_Size_USD
0,BP,ST,2019-01-11,Sale,NaN,2019-03-04,15001.0
1,GCGMF,ST,2019-01-11,Sale,NaN,2019-03-04,1001.0
2,GCGMF,ST,2019-01-11,Sale,NaN,2019-03-04,1001.0
3,GCGMF,ST,2019-01-11,Sale,NaN,2019-03-04,1001.0
4,DELL,ST,2019-01-14,Purchase,NaN,2019-03-04,1001.0
...,...,...,...,...,...,...,...
1271,FLL,ST,2025-03-20,Sale,NaN,2025-04-11,15001.0
1272,FLL,ST,2025-03-28,Sale,NaN,2025-04-11,1001.0
1273,FLL,ST,2025-06-13,Sale,NaN,2025-07-10,250001.0
1274,FLL,ST,2025-06-13,Sale,NaN,2025-07-10,500001.0


In [108]:
congressmen[9].positions_df

,Date,Ticker,Amount,Filed,Trade_Size_USD
0,2019-01-11,BP,-15001.0,2019-03-04,15001.0
1,2019-01-11,GCGMF,-1001.0,2019-03-04,1001.0
2,2019-01-11,GCGMF,-1001.0,2019-03-04,1001.0
3,2019-01-11,GCGMF,-1001.0,2019-03-04,1001.0
4,2019-01-14,DELL,1001.0,2019-03-04,1001.0
...,...,...,...,...,...
1270,2025-03-20,FLL,-15001.0,2025-04-11,15001.0
1271,2025-03-28,FLL,-1001.0,2025-04-11,1001.0
1272,2025-06-13,FLL,-500001.0,2025-07-10,500001.0
1273,2025-06-13,FLL,-250001.0,2025-07-10,250001.0


In [109]:
all_tickers.add('SPY')
prices_df = fetch_prices(list(all_tickers))

Fetching prices for 1302 tickers...


C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True


1 Failed download:
['WRK']: YFTzMissingError('possibly delisted; no timezone found')
C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipyk

  Fetched 50/1302 tickers


C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True


1 Failed download:
['PER']: YFTzMissingError('possibly delisted; no timezone found')
C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True


1 Failed download:
['UTX']: YFTzMissingError('possibly delisted; no timezone found')
C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() ha

  Fetched 100/1302 tickers


...

  Fetched 1250/1302 tickers


C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True


1 Failed download:
['VSNT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-12-04 00:00:00 -> 2025-01-07 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1543899600, endDate = 1736226000")')
C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykern

  Fetched 1300/1302 tickers


C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\79944939.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True


1 Failed download:
['BHI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-12-04 00:00:00 -> 2025-01-07 00:00:00)')


Failed to fetch: 236 tickers
Successfully fetched prices for 1066 tickers


In [110]:
prices_df['T']

Date
2018-12-04    13.844340
2018-12-06    13.754235
2018-12-07    13.578538
2018-12-10    13.474916
2018-12-11    13.380311
                ...    
2024-12-30    21.389805
2024-12-31    21.541172
2025-01-02    21.597935
2025-01-03    21.446569
2025-01-06    21.380346
Name: T, Length: 1531, dtype: float64

In [111]:
 spy_returns = get_spy_returns()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\3416171390.py:3: FutureWarning:

YF.download() has changed argument auto_adjust default to True



In [112]:
portfolios = {}
copycat_objects = {}

for congressman in congressmen:
    if congressman.transactions is None:
        continue

    print(f"  Simulating: {congressman.name}")
    copycat = CopycatPortfolio(congressman, prices_df)
    result = copycat.simulate()

    if result is not None and len(result) > 0:
        portfolios[congressman.name] = result
        copycat_objects[congressman.name] = copycat

  Simulating: Josh Gottheimer
    Initial positions: 70 tickers
    Transactions in range: 2506
  Simulating: Nancy Pelosi
    Initial positions: 1 tickers
    Transactions in range: 44
  Simulating: Kevin Hern
    Initial positions: 1 tickers
    Transactions in range: 659
  Simulating: Mark Green
    Transactions in range: 618
  Simulating: Bradley S. Schneider
    Initial positions: 5 tickers
    Transactions in range: 50
  Simulating: Virginia Foxx
    Initial positions: 14 tickers
    Transactions in range: 610
  Simulating: Kim Schrier
    Transactions in range: 41
  Simulating: Mikie Sherrill
    Transactions in range: 292
  Simulating: Daniel Meuser
    Transactions in range: 59
  Simulating: Susie Lee
    Transactions in range: 1257
  Simulating: John W. Rose
    Transactions in range: 9
  Simulating: Brian Mast
    Initial positions: 2 tickers
    Transactions in range: 83
  Simulating: Carol Devine Miller
    Transactions in range: 103
  Simulating: Michael C. Burgess
    Tr

In [113]:
copycat_objects

{'Josh Gottheimer': <__main__.CopycatPortfolio at 0x1601f2c5c90>,
 'Nancy Pelosi': <__main__.CopycatPortfolio at 0x1601e0c0210>,
 'Kevin Hern': <__main__.CopycatPortfolio at 0x16060c9f550>,
 'Mark Green': <__main__.CopycatPortfolio at 0x1606e0622d0>,
 'Bradley S. Schneider': <__main__.CopycatPortfolio at 0x1605c597210>,
 'Virginia Foxx': <__main__.CopycatPortfolio at 0x1605f9bc110>,
 'Kim Schrier': <__main__.CopycatPortfolio at 0x1605c9ef1d0>,
 'Mikie Sherrill': <__main__.CopycatPortfolio at 0x1605ca5c2d0>,
 'Daniel Meuser': <__main__.CopycatPortfolio at 0x1605c909650>,
 'Susie Lee': <__main__.CopycatPortfolio at 0x1601e049350>,
 'John W. Rose': <__main__.CopycatPortfolio at 0x1605fbffb90>,
 'Brian Mast': <__main__.CopycatPortfolio at 0x1606e0380d0>,
 'Carol Devine Miller': <__main__.CopycatPortfolio at 0x1607eced390>,
 'Michael C. Burgess': <__main__.CopycatPortfolio at 0x1601f1ca6d0>,
 'Pete Sessions': <__main__.CopycatPortfolio at 0x16060c3a5d0>,
 'Lois Frankel': <__main__.CopycatPo

In [114]:
portfolios['Nancy Pelosi']

,Date,stock_value,cash,cash_invested,portfolio_value,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,2.500010e+05,1.000000e+05,0.000000e+00,3.500010e+05,0.000000,0.000000,0.000000,0.000000
1,2019-01-04,2.564243e+05,1.000000e+05,0.000000e+00,3.564243e+05,0.000000,0.000000,0.025369,0.025369
2,2019-01-07,2.610727e+05,1.000000e+05,0.000000e+00,3.610727e+05,0.000000,0.000000,0.017966,0.043334
3,2019-01-08,2.643688e+05,1.000000e+05,0.000000e+00,3.643688e+05,0.000000,0.000000,0.012546,0.055880
4,2019-01-09,2.600087e+05,1.000000e+05,0.000000e+00,3.600087e+05,0.000000,0.000000,-0.016630,0.039250
...,...,...,...,...,...,...,...,...,...
1561,2024-12-27,6.485809e+06,-4.913048e+06,5.013048e+06,1.572762e+06,1.243916,124.391563,-0.015414,3.255907
1562,2024-12-30,6.416951e+06,-4.913048e+06,5.013048e+06,1.503903e+06,1.230180,123.017980,-0.010674,3.245233
1563,2024-12-31,6.379285e+06,-4.913048e+06,5.013048e+06,1.466238e+06,1.222666,122.266628,-0.005887,3.239346
1564,2025-01-01,6.379285e+06,-4.913048e+06,5.013048e+06,1.466238e+06,1.222666,122.266628,0.000000,3.239346


In [115]:
congressmen[1].get_position("2020-07-16")

{'PYPL': 1000002.0, 'T': 250001.0, 'WORK': 250001.0}

## Visualization

In [45]:
def plot_individual_returns(portfolio_df, spy_df, legislator_name, save_path=None):
    """Plot individual portfolio returns vs SPY."""
    fig = go.Figure()

    # Portfolio returns
    fig.add_trace(go.Scatter(
        x=portfolio_df['Date'],
        y=portfolio_df['cumulative_return_pct'],
        mode='lines',
        name=legislator_name,
        line=dict(width=2)
    ))

    # SPY returns
    fig.add_trace(go.Scatter(
        x=spy_df['Date'],
        y=spy_df['cumulative_return_pct'],
        mode='lines',
        name='SPY',
        line=dict(width=2, dash='dash', color='gray')
    ))

    fig.update_layout(
        title=f'{legislator_name} Portfolio vs SPY',
        xaxis_title='Date',
        yaxis_title='Cumulative Return (%)',
        template='plotly_white',
        legend=dict(x=0.02, y=0.98),
        hovermode='x unified'
    )

    if save_path:
        fig.write_html(save_path)

    return fig

In [46]:
def plot_all_portfolios(portfolios_dict, spy_df, save_path=None):
    """Plot all portfolios comparison."""
    fig = go.Figure()

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
              '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
              '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5']

    for i, (name, df) in enumerate(portfolios_dict.items()):
        fig.add_trace(go.Scatter(
            x=df['Date'],
            y=df['cumulative_return_pct'],
            mode='lines',
            name=name,
            line=dict(width=1.5, color=colors[i % len(colors)])
        ))

    # SPY
    fig.add_trace(go.Scatter(
        x=spy_df['Date'],
        y=spy_df['cumulative_return_pct'],
        mode='lines',
        name='SPY',
        line=dict(width=3, color='black', dash='dash')
    ))

    fig.update_layout(
        title='All Copy-Cat Portfolios vs SPY',
        xaxis_title='Date',
        yaxis_title='Cumulative Return (%)',
        template='plotly_white',
        legend=dict(x=1.02, y=1, font=dict(size=9)),
        hovermode='x unified',
        height=700,
        width=1200
    )

    if save_path:
        fig.write_html(save_path)

    return fig

In [47]:
def plot_position_pie(holdings_dict, date_str, legislator_name, save_path=None):
    """Plot pie chart of portfolio positions."""
    if not holdings_dict:
        return None

    # Sort by value and take top 10
    sorted_holdings = sorted(holdings_dict.items(), key=lambda x: x[1], reverse=True)

    if len(sorted_holdings) > 10:
        top_10 = sorted_holdings[:10]
        other_value = sum(v for _, v in sorted_holdings[10:])
        top_10.append(('Other', other_value))
        sorted_holdings = top_10

    labels = [h[0] for h in sorted_holdings]
    values = [h[1] for h in sorted_holdings]

    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        textinfo='label+percent',
        textposition='outside'
    )])

    fig.update_layout(
        title=f'{legislator_name} Portfolio - {date_str}',
        template='plotly_white'
    )

    if save_path:
        fig.write_html(save_path)

    return fig



In [48]:
def plot_all_snapshots_for_legislator(copycat_portfolio, legislator_name, save_dir):
    """Generate all pie chart snapshots for a legislator."""
    for date_str, holdings in copycat_portfolio.snapshots.items():
        if holdings:
            safe_name = legislator_name.replace(" ", "_").replace(".", "")
            filename = f"{safe_name}_position_{date_str.replace('-', '')}.html"
            save_path = os.path.join(save_dir, filename)
            plot_position_pie(holdings, date_str, legislator_name, save_path)



In [49]:
    # Individual return plots
    individual_dir = os.path.join(OUTPUT_DIR, 'individual_returns')
    os.makedirs(individual_dir, exist_ok=True)

    for name, portfolio_df in portfolios.items():
        safe_name = name.replace(" ", "_").replace(".", "")
        save_path = os.path.join(individual_dir, f"{safe_name}_returns.html")
        plot_individual_returns(portfolio_df, spy_returns, name, save_path)

    # Combined comparison plot
    combined_path = os.path.join(OUTPUT_DIR, 'all_portfolios_comparison.html')
    plot_all_portfolios(portfolios, spy_returns, save_path=combined_path)

    # Pie chart snapshots
    print("  Generating position pie charts...")
    pie_dir = os.path.join(OUTPUT_DIR, 'position_snapshots')
    os.makedirs(pie_dir, exist_ok=True)

    for name, copycat in copycat_objects.items():
        plot_all_snapshots_for_legislator(copycat, name, pie_dir)

  Generating position pie charts...


In [50]:
congressmen[1].name

'Nancy Pelosi'

In [51]:
congressmen[1].transactions.head(30)


,Ticker,TickerType,Traded,Transaction,Description,Filed,Trade_Size_USD
0,T,ST,2018-10-24,Purchase,"PURCHASED 10,000 SHARES.",2018-11-08,250001.0
1,BFET,ST,2019-02-07,Sale,FINAL CASH LIQUIDATION INITIATED BY CORPORATION.,2019-03-11,1001.0
2,V,ST,2019-08-07,Sale,"SALE OF 1,000 SHARES",2019-09-18,100001.0
3,V,ST,2019-08-07,Sale,SALE OF 1.000 SHARES,2019-09-18,100001.0
4,WORK,ST,2020-02-20,Purchase,"PURCHASED 10,000 SHARES.",2020-04-01,250001.0
5,AAPL,ST,2020-05-08,Sale,"SOLD 3,300 SHARES.",2020-06-09,1000001.0
6,V,ST,2020-05-08,Sale,"SOLD 3,000 SHARES.",2020-06-09,500001.0
7,AAPL,ST,2020-05-08,Sale,"SOLD 3,000 SHARES.",2020-06-09,500001.0
8,AAPL,ST,2020-05-08,Sale,"SOLD 1,700 SHARES.",2020-06-09,500001.0
9,FB,ST,2020-05-08,Sale,"SOLD 5,000 SHARES.",2020-06-09,1000001.0


In [52]:
wyden = Congressman_df(
    name="Ron Wyden",
    party="Democrat",
    chamber="Senate",
    state="OR",
    bioguide="W000779",
    trades_df=raw_df
)

In [53]:
composition = wyden.get_portfolio_composition('2020-07-16', prices_df)
print(composition)


Portfolio Composition for Ron Wyden
As of: 2020-07-16
Total Value: $2,084,088.00
Number of Positions: 29

   Ticker  Cost_Basis_USD        Shares       Price     Value  Weight
0    NVDA        245006.0  24257.249296   10.100321  245006.0   11.76
1    AMZN        220010.0   1466.782274  149.994995  220010.0   10.56
2     DIS        203007.0   1739.828277  116.682205  203007.0    9.74
3    AAPL        160006.0   1708.928994   93.629402  160006.0    7.68
4    PYPL        130004.0    767.354893  169.418350  130004.0    6.24
5     VMC        115003.0    968.647765  118.725304  115003.0    5.52
6     MAR        115003.0   1280.723129   89.795364  115003.0    5.52
7    GOOG         97006.0   1287.662054   75.334984   97006.0    4.65
8    AMAT         75005.0   1264.288314   59.325867   75005.0    3.60
9    MSFT         75005.0    385.257883  194.687775   75005.0    3.60
10   KLAC         75005.0    407.091972  184.245834   75005.0    3.60
11    STZ         65002.0    395.247815  164.458847  

In [54]:
composition = wyden.get_portfolio_composition('2020-05-01', prices_df)
print(composition)


Portfolio Composition for Ron Wyden
As of: 2020-05-01
Total Value: $630,028.00
Number of Positions: 18

   Ticker  Cost_Basis_USD       Shares       Price     Value  Weight
0    AMZN        130004.0  1137.372905  114.302002  130004.0   20.63
1     DIS        100002.0   970.208473  103.072693  100002.0   15.87
2    GOOG         52003.0   793.467207   65.538940   52003.0    8.25
3     VMC         50001.0   474.296996  105.421288   50001.0    7.94
4    NVDA         50001.0  7100.124720    7.042271   50001.0    7.94
5    PYPL         50001.0   415.525983  120.331825   50001.0    7.94
6    AMAT         30002.0   675.023009   44.445892   30002.0    4.76
7    KLAC         30002.0   213.460934  140.550308   30002.0    4.76
8       T         16002.0  1051.732248   15.214899   16002.0    2.54
9    SBUX         16002.0   246.561069   64.900757   16002.0    2.54
10      V         15001.0    89.013245  168.525482   15001.0    2.38
11    UPS         15001.0   205.297150   73.069695   15001.0    2.3

In [55]:
copycat_wyden = CopycatPortfolio(wyden, prices_df)
copycat_wyden.simulate()

    Transactions in range: 241


,Date,stock_value,cash,cash_invested,portfolio_value,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,0.000000e+00,1.000000e+05,0.000000e+00,1.000000e+05,0.000000,0.000000,0.000000,0.000000
1,2019-01-04,0.000000e+00,1.000000e+05,0.000000e+00,1.000000e+05,0.000000,0.000000,0.000000,0.000000
2,2019-01-07,0.000000e+00,1.000000e+05,0.000000e+00,1.000000e+05,0.000000,0.000000,0.000000,0.000000
3,2019-01-08,0.000000e+00,1.000000e+05,0.000000e+00,1.000000e+05,0.000000,0.000000,0.000000,0.000000
4,2019-01-09,0.000000e+00,1.000000e+05,0.000000e+00,1.000000e+05,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
1561,2024-12-27,9.247200e+06,-2.113034e+06,2.213034e+06,7.134166e+06,4.178517,417.851730,-0.017513,2.687267
1562,2024-12-30,9.224758e+06,-2.113034e+06,2.213034e+06,7.111724e+06,4.168376,416.837643,-0.002430,2.684837
1563,2024-12-31,9.078265e+06,-2.113034e+06,2.213034e+06,6.965231e+06,4.102181,410.218109,-0.016008,2.668829
1564,2025-01-01,9.078265e+06,-2.113034e+06,2.213034e+06,6.965231e+06,4.102181,410.218109,0.000000,2.668829


In [56]:
breakdown_wyden = copycat_wyden.get_portfolio_breakdown('2020-05-01')
print(breakdown_wyden)


Portfolio Breakdown for Ron Wyden
As of: 2020-05-01
Total Portfolio Value: $-250,354.12
  - Stock Holdings:    $1,862,679.58 (-744.0%)
  - Cash:              $-2,113,033.70 (844.0%)
Number of Positions:   27

   Asset   Type        Shares       Price         Value  Weight
0   NVDA  Stock  40970.546925    7.042271  2.885257e+05 -115.25
1   AAPL  Stock   3249.534588   69.912155  2.271820e+05  -90.74
2   AMZN  Stock   1934.177956  114.302002  2.210804e+05  -88.31
3   MSFT  Stock   1165.340977  166.203690  1.936840e+05  -77.36
4    DIS  Stock   1531.427024  103.072693  1.578483e+05  -63.05
5    VMC  Stock   1028.715137  105.421288  1.084485e+05  -43.32
6   GOOG  Stock   1653.035570   65.538940  1.083382e+05  -43.27
7    UPS  Stock   1109.227053   73.069695  8.105088e+04  -32.37
8    PSA  Stock    504.111613  144.562210  7.287549e+04  -29.11
9    STX  Stock   1394.564184   39.070877  5.448685e+04  -21.76
10   DHI  Stock   1242.880670   43.755558  5.438294e+04  -21.72
11   DPZ  Stock    144

In [57]:
breakdown_wyden = copycat_wyden.get_portfolio_breakdown('2020-07-01')
print(breakdown_wyden)


Portfolio Breakdown for Ron Wyden
As of: 2020-07-01
Total Portfolio Value: $105,393.58
  - Stock Holdings:    $2,218,427.28 (2104.9%)
  - Cash:              $-2,113,033.70 (-2004.9%)
Number of Positions:   27

   Asset   Type        Shares       Price         Value   Weight
0   NVDA  Stock  40970.546925    9.497627  3.891230e+05   369.21
1   AAPL  Stock   3249.534588   88.299126  2.869311e+05   272.25
2   AMZN  Stock   1934.177956  143.934998  2.783959e+05   264.15
3   MSFT  Stock   1165.340977  195.432480  2.277455e+05   216.09
4    DIS  Stock   1531.427024  110.409912  1.690847e+05   160.43
5   GOOG  Stock   1653.035570   71.366737  1.179718e+05   111.93
6    VMC  Stock   1028.715137  110.821747  1.140040e+05   108.17
7    UPS  Stock   1109.227053   91.929504  1.019707e+05    96.75
8    PSA  Stock    504.111613  157.777863  7.953765e+04    75.47
9    DHI  Stock   1242.880670   50.826218  6.317092e+04    59.94
10   STX  Stock   1394.564184   39.236065  5.471721e+04    51.92
11   DPZ 

In [58]:
breakdown_wyden = copycat_wyden.get_portfolio_breakdown('2021-12-01')
print(breakdown_wyden)


Portfolio Breakdown for Ron Wyden
As of: 2021-12-01
Total Portfolio Value: $2,131,627.13
  - Stock Holdings:    $4,244,660.83 (199.1%)
  - Cash:              $-2,113,033.70 (-99.1%)
Number of Positions:   27

   Asset   Type        Shares       Price         Value  Weight
0   NVDA  Stock  40970.546925   31.374226  1.285419e+06   60.30
1   AAPL  Stock   3249.534588  161.384567  5.244247e+05   24.60
2   MSFT  Stock   1165.340977  319.379730  3.721863e+05   17.46
3   AMZN  Stock   1934.177956  172.186005  3.330384e+05   15.62
4   GOOG  Stock   1653.035570  140.563736  2.323569e+05   10.90
5    DIS  Stock   1531.427024  138.879456  2.126838e+05    9.98
6    VMC  Stock   1028.715137  182.988281  1.882428e+05    8.83
7    UPS  Stock   1109.227053  161.337112  1.789595e+05    8.40
8    PSA  Stock    504.111613  268.371613  1.352892e+05    6.35
9    STX  Stock   1394.564184   90.405411  1.260761e+05    5.91
10   DHI  Stock   1242.880670   93.360489  1.160359e+05    5.44
11     F  Stock   6813

# Portfolio optimization

In [116]:
def get_risk_free_rate():
    """Fetch 3-month US Treasury Bill rate from FRED via yfinance."""
    try:
        # ^IRX is the 13-week Treasury Bill rate
        tbill = yf.download('^IRX', period='5d', progress=False)
        if not tbill.empty:
            rate = tbill['Close'].iloc[-1].item() / 100  # Convert to decimal
            return rate
    except Exception as e:
        print(f"Could not fetch T-bill rate: {e}")
    # Default fallback
    return 0.045  # 4.5% as fallback


def get_top_n_stocks_at_date(congressmen_list, as_of_date, n=20):
    """
    Get top N stocks held by congressmen as of a given date.
    Uses Filed date for visibility (simulating real-world delay).
    
    Args:
        congressmen_list: List of Congressman_df instances
        as_of_date: Date to check (only filings up to this date are visible)
        n: Number of top stocks to return
        
    Returns:
        List of (ticker, total_trade_size_usd) tuples, sorted descending
    """
    as_of = pd.to_datetime(as_of_date)
    all_holdings = {}
    
    for congressman in congressmen_list:
        if congressman.positions_df.empty:
            continue
        
        # Filter by Filed date <= as_of_date
        df = congressman.positions_df.copy()
        df['Filed'] = pd.to_datetime(df['Filed'])
        df = df[df['Filed'] <= as_of]
        
        if df.empty:
            continue
        
        # Aggregate Trade_Size_USD by ticker (absolute value for total exposure)
        for _, row in df.iterrows():
            ticker = row['Ticker']
            trade_size = abs(row.get('Trade_Size_USD', 0))
            if pd.notna(trade_size) and trade_size > 0:
                all_holdings[ticker] = all_holdings.get(ticker, 0) + trade_size
    
    # Sort by total trade size and return top N
    sorted_stocks = sorted(all_holdings.items(), key=lambda x: x[1], reverse=True)
    return sorted_stocks[:n]


def get_rebalance_dates(start_date, end_date, freq='quarterly'):
    """
    Generate rebalancing dates based on frequency.
    Adjusts non-business days to nearest earlier business day.
    
    Args:
        start_date, end_date: Date range
        freq: 'weekly', 'bi-weekly', 'monthly', 'quarterly', 'annually'
        
    Returns:
        List of rebalance dates (all business days)
    """
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    
    freq_map = {
        'weekly': 'W-FRI',
        'bi-weekly': '2W-FRI',
        'monthly': 'ME',
        'quarterly': 'QE',
        'annually': 'YE'
    }
    
    pandas_freq = freq_map.get(freq, 'QE')
    dates = pd.date_range(start=start_dt, end=end_dt, freq=pandas_freq)
    
    # Ensure start date is included as first rebalance
    if len(dates) == 0 or dates[0] > start_dt:
        dates = pd.DatetimeIndex([start_dt]).append(dates)
    
    # Adjust to nearest earlier business day if not a business day
    adjusted_dates = []
    for d in dates:
        # If weekend, roll back to Friday
        if d.weekday() == 5:  # Saturday
            d = d - timedelta(days=1)
        elif d.weekday() == 6:  # Sunday
            d = d - timedelta(days=2)
        adjusted_dates.append(d)
    
    return adjusted_dates



In [117]:
class CustomPortfolio:
    """
    Builds optimized portfolio from top N congressional holdings.
    
    Weighting schemes:
    - equal_weight: Equal number of shares per stock
    - equal_value: Equal dollar value per stock (1/N of capital each)
    - mean_variance: Optimize using historical returns/covariance
    
    Return Calculation:
    - daily_return = stock_value.pct_change()
    - On rebalance days, invest exactly prev_stock_value to ensure continuity
    - cumulative_return = (1 + daily_return).cumprod() - 1
    """
    
    def __init__(self, congressmen_list, prices_df, 
                 rebalance_freq='quarterly', n_stocks=20,
                 lookback_months=12, weighting='equal_value'):
        self.congressmen_list = congressmen_list
        self.prices_df = prices_df
        self.rebalance_freq = rebalance_freq
        self.n_stocks = n_stocks
        self.lookback_months = lookback_months
        self.weighting = weighting
        
        self.holdings = {}  # {ticker: shares}
        self.holdings_history = {}  # {date: holdings_dict}
        self.history = None
        self.rebalance_log = []
    
    def simulate(self, start_date=CONGRESS_START_DATE, end_date=CONGRESS_END_DATE, debug=False):
        """Run day-by-day simulation with periodic rebalancing."""
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        
        # Reset state
        self.holdings = {}
        self.holdings_history = {}
        self.rebalance_log = []
        
        # Get rebalance dates
        rebalance_dates = get_rebalance_dates(start_date, end_date, self.rebalance_freq)
        rebalance_set = set(d.date() for d in rebalance_dates)
        
        if debug:
            print(f"Rebalance dates: {len(rebalance_dates)}")
            print(f"First 5: {[d.strftime('%Y-%m-%d') for d in rebalance_dates[:5]]}")
        
        # Generate daily values
        date_range = pd.date_range(start=start_dt, end=end_dt, freq='B')
        
        records = []
        prev_stock_value = None
        
        for date in date_range:
            date_key = date.date()
            
            # Rebalance on rebalance dates
            if date_key in rebalance_set:
                # Amount to invest = previous day's stock value (for continuity)
                # On first rebalance, use INITIAL_CAPITAL
                amount_to_invest = prev_stock_value if prev_stock_value is not None else INITIAL_CAPITAL
                
                if debug:
                    print(f"REBALANCE {date.strftime('%Y-%m-%d')}: investing ${amount_to_invest:,.0f}")
                
                self._rebalance(date, amount_to_invest)
            
            # Calculate stock holdings value
            stock_value = self._calculate_stock_value(date)
            
            # Store holdings snapshot for this date
            self.holdings_history[date] = dict(self.holdings)
            
            records.append({
                'Date': date,
                'stock_value': stock_value,
                'n_holdings': len(self.holdings)
            })
            
            prev_stock_value = stock_value
        
        # Create DataFrame and calculate returns
        df = pd.DataFrame(records)
        
        if not df.empty:
            # Daily return = percentage change of stock_value
            df['daily_return'] = df['stock_value'].pct_change().fillna(0)
            
            # Cumulative return = cumprod of (1 + daily_return) - 1
            df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1
            df['cumulative_return_pct'] = df['cumulative_return'] * 100
            
            # Log returns
            df['log_return'] = np.log(1 + df['daily_return']).replace([np.inf, -np.inf], 0).fillna(0)
            df['cumulative_log_return'] = df['log_return'].cumsum()
        
        self.history = df
        return df
    
    def _rebalance(self, date, amount_to_invest):
        """Rebalance portfolio: sell all, buy new top N stocks.
        
        Key: invest exactly amount_to_invest (= prev day's stock value)
        to ensure stock_value continuity across rebalance.
        """
        # Get top N stocks
        top_stocks = get_top_n_stocks_at_date(self.congressmen_list, date, self.n_stocks)
        
        if not top_stocks:
            return
        
        tickers = [t[0] for t in top_stocks]
        
        # Clear holdings
        self.holdings = {}
        
        # Get weights
        weights = self._get_weights(tickers, date)
        
        # Buy new positions
        for ticker, weight in zip(tickers, weights):
            if weight <= 0:
                continue
            
            price = get_price_on_date(self.prices_df, ticker, date)
            if price is None or price <= 0:
                continue
            
            dollars = amount_to_invest * weight
            shares = dollars / price
            self.holdings[ticker] = shares
        
        self.rebalance_log.append({
            'date': date,
            'amount_invested': amount_to_invest,
            'n_stocks': len(self.holdings),
            'tickers': list(self.holdings.keys())
        })
    
    def _get_weights(self, tickers, date):
        """Calculate portfolio weights based on weighting scheme.
        Only includes tickers that have valid prices in prices_df.
        """
        # First, filter to tickers that have valid prices
        valid_tickers = []
        valid_prices = []
        
        for t in tickers:
            p = get_price_on_date(self.prices_df, t, date)
            if p is not None and p > 0:
                valid_tickers.append(t)
                valid_prices.append(p)
        
        n = len(valid_tickers)
        if n == 0:
            return [0] * len(tickers)  # No valid tickers
        
        if self.weighting == 'equal_weight':
            # Equal number of shares - weight proportional to 1/price
            inv_prices = [1/p for p in valid_prices]
            total = sum(inv_prices)
            valid_weights = [ip / total for ip in inv_prices]
        
        elif self.weighting == 'equal_value':
            # Equal dollar value per stock
            valid_weights = [1 / n] * n
        
        elif self.weighting == 'mean_variance':
            return self._calculate_mean_variance_weights(tickers, date)
        
        else:
            valid_weights = [1 / n] * n
        
        # Map back to original ticker list (0 weight for missing tickers)
        weight_map = dict(zip(valid_tickers, valid_weights))
        return [weight_map.get(t, 0) for t in tickers]
    
    def _calculate_mean_variance_weights(self, tickers, date):
        """Calculate mean-variance optimal weights (minimum variance)."""
        n = len(tickers)
        
        # Get historical returns
        end_date = pd.to_datetime(date)
        start_date = end_date - timedelta(days=self.lookback_months * 30)
        
        returns_data = []
        valid_tickers = []
        
        for ticker in tickers:
            if ticker not in self.prices_df.columns:
                continue
            
            prices = self.prices_df[ticker].loc[start_date:end_date].dropna()
            if len(prices) < 20:
                continue
            
            returns = prices.pct_change().dropna()
            if len(returns) > 0:
                returns_data.append(returns)
                valid_tickers.append(ticker)
        
        if len(valid_tickers) < 2:
            return [1 / n] * n
        
        # Align returns
        returns_df = pd.DataFrame({t: r for t, r in zip(valid_tickers, returns_data)})
        returns_df = returns_df.dropna()
        
        if len(returns_df) < 2:
            return [1 / n] * n
        
        # Covariance matrix
        cov_matrix = returns_df.cov().values
        n_valid = len(valid_tickers)
        
        # Minimize variance
        def portfolio_variance(w):
            return w @ cov_matrix @ w
        
        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
        bounds = [(0, 0.25) for _ in range(n_valid)]
        x0 = np.array([1 / n_valid] * n_valid)
        
        try:
            result = minimize(portfolio_variance, x0, method='SLSQP',
                            bounds=bounds, constraints=constraints)
            optimized = result.x if result.success else x0
        except:
            optimized = x0
        
        # Map back to original tickers
        ticker_weights = dict(zip(valid_tickers, optimized))
        weights = [ticker_weights.get(t, 0) for t in tickers]
        
        # Normalize
        total = sum(weights)
        if total > 0:
            weights = [w / total for w in weights]
        else:
            weights = [1 / n] * n
        
        return weights
    
    def _calculate_stock_value(self, date):
        """Calculate total value of stock holdings."""
        total = 0
        for ticker, shares in self.holdings.items():
            price = get_price_on_date(self.prices_df, ticker, date)
            if price is not None and price > 0:
                total += shares * price
        return total
    
    def get_portfolio_composition(self, date=None):
        """
        Get detailed portfolio composition at a specific date.
        Uses holdings_history from simulation if date is provided.
        
        Returns:
            DataFrame with: Asset, Type, Shares, Price, Value, Weight (%)
        """
        if date:
            as_of = pd.to_datetime(date)
            # Look up holdings from history for this date
            # Find the closest date in holdings_history that is <= as_of
            available_dates = sorted(self.holdings_history.keys())
            holdings_to_use = {}
            for d in available_dates:
                if d <= as_of:
                    holdings_to_use = self.holdings_history[d]
                else:
                    break
            if not holdings_to_use and available_dates:
                # If date is before first rebalance, use first available
                holdings_to_use = self.holdings_history[available_dates[0]]
        else:
            as_of = None
            holdings_to_use = self.holdings  # Use current holdings
        
        records = []
        total_stock_value = 0
        missing_tickers = []
        
        # Add stock holdings
        for ticker, shares in holdings_to_use.items():
            if as_of:
                price = get_price_on_date(self.prices_df, ticker, as_of)
            else:
                if ticker in self.prices_df.columns:
                    prices = self.prices_df[ticker].dropna()
                    price = prices.iloc[-1] if not prices.empty else None
                else:
                    price = None
            
            if price is not None and price > 0:
                value = shares * price
                total_stock_value += value
                records.append({
                    'Asset': ticker, 'Type': 'Stock',
                    'Shares': shares, 'Price': price, 'Value': value
                })
            else:
                missing_tickers.append(ticker)
                records.append({
                    'Asset': ticker, 'Type': 'Stock',
                    'Shares': shares, 'Price': None, 'Value': 0
                })
        
        df = pd.DataFrame(records)
        
        # Calculate weights based on total stock value
        if not df.empty and total_stock_value > 0:
            df['Weight (%)'] = (df['Value'] / total_stock_value * 100).round(2)
        elif not df.empty:
            df['Weight (%)'] = 0
        
        if not df.empty:
            df = df.sort_values('Value', ascending=False).reset_index(drop=True)
        
        # Print summary
        date_str = as_of.strftime('%Y-%m-%d') if as_of else 'Current'
        print(f"\n{'='*70}")
        print(f"Portfolio Composition - {date_str}")
        print(f"Weighting: {self.weighting}, Rebalance: {self.rebalance_freq}")
        print(f"{'='*70}")
        print(f"Total Value:         ${total_stock_value:,.2f}")
        print(f"Number of Positions: {len(holdings_to_use)}")
        if missing_tickers:
            print(f"Missing Prices:      {len(missing_tickers)} - {missing_tickers[:5]}")
        print(f"{'='*70}\n")
        
        return df


## Equal weighted portfolio

In [118]:
equal_weight = CustomPortfolio(congressmen, prices_df, weighting='equal_weight')
equal_weight.simulate()

,Date,stock_value,n_holdings,daily_return,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,100000.000000,18,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
1,2019-01-04,104517.812125,18,4.517812e-02,0.045178,4.517812,4.418732e-02,0.044187
2,2019-01-07,107882.919141,18,3.219649e-02,0.078829,7.882919,3.168905e-02,0.075876
3,2019-01-08,109324.779154,18,1.336504e-02,0.093248,9.324779,1.327652e-02,0.089153
4,2019-01-09,110449.385820,18,1.028684e-02,0.104494,10.449386,1.023429e-02,0.099387
...,...,...,...,...,...,...,...,...
1561,2024-12-27,711259.604377,14,4.878807e-04,6.112596,611.259604,4.877617e-04,1.961867
1562,2024-12-30,716110.264920,14,6.819817e-03,6.161103,616.110265,6.796668e-03,1.968664
1563,2024-12-31,716110.264920,14,2.220446e-16,6.161103,616.110265,2.220446e-16,1.968664
1564,2025-01-01,716110.264920,14,0.000000e+00,6.161103,616.110265,0.000000e+00,1.968664


## Equal market cap portfolio

In [119]:
equal_value = CustomPortfolio(congressmen, prices_df, weighting='equal_value')
equal_value.simulate()

,Date,stock_value,n_holdings,daily_return,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,100000.000000,18,0.000000,0.000000,0.000000,0.000000,0.000000
1,2019-01-04,104042.829975,18,0.040428,0.040428,4.042830,0.039632,0.039632
2,2019-01-07,106630.207679,18,0.024868,0.066302,6.630208,0.024564,0.064197
3,2019-01-08,107927.161635,18,0.012163,0.079272,7.927162,0.012090,0.076286
4,2019-01-09,109156.641375,18,0.011392,0.091566,9.156641,0.011327,0.087614
...,...,...,...,...,...,...,...,...
1561,2024-12-27,555103.588374,14,-0.010691,4.551036,455.103588,-0.010749,1.713985
1562,2024-12-30,552387.035831,14,-0.004894,4.523870,452.387036,-0.004906,1.709079
1563,2024-12-31,552387.035831,14,0.000000,4.523870,452.387036,0.000000,1.709079
1564,2025-01-01,552387.035831,14,0.000000,4.523870,452.387036,0.000000,1.709079


## Mean-Variance portfolio

In [120]:
mean_var = CustomPortfolio(congressmen, prices_df, weighting='mean_variance')
mean_var.simulate()

,Date,stock_value,n_holdings,daily_return,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,100000.000000,16,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
1,2019-01-04,102837.090415,16,2.837090e-02,0.028371,2.837090,2.797590e-02,0.027976
2,2019-01-07,103573.743915,16,7.163306e-03,0.035737,3.573744,7.137771e-03,0.035114
3,2019-01-08,104888.857624,16,1.269737e-02,0.048889,4.888858,1.261743e-02,0.047731
4,2019-01-09,104584.891012,16,-2.897988e-03,0.045849,4.584891,-2.902195e-03,0.044829
...,...,...,...,...,...,...,...,...
1561,2024-12-27,477306.238869,14,-1.069120e-02,3.773062,377.306239,-1.074877e-02,1.562988
1562,2024-12-30,474970.409117,14,-4.893776e-03,3.749704,374.970409,-4.905790e-03,1.558082
1563,2024-12-31,474970.409117,14,6.661338e-16,3.749704,374.970409,6.661338e-16,1.558082
1564,2025-01-01,474970.409117,14,0.000000e+00,3.749704,374.970409,0.000000e+00,1.558082


## Metrics and Comparison

In [121]:
def calculate_portfolio_metrics(history_df, risk_free_rate=None):
    """
    Calculate portfolio performance metrics.
    
    Returns:
        dict with: annualized_return, volatility, sharpe_ratio, max_drawdown
    """
    if risk_free_rate is None:
        risk_free_rate = get_risk_free_rate()
    
    if history_df is None or history_df.empty:
        return {
            'annualized_return': 0,
            'volatility': 0,
            'sharpe_ratio': 0,
            'max_drawdown': 0
        }
    
    # Daily returns from log returns
    if 'log_return' in history_df.columns:
        daily_returns = history_df['log_return'].dropna()
    else:
        daily_returns = history_df['portfolio_value'].pct_change().dropna()
    
    # Annualized metrics (252 trading days)
    mean_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()
    
    annualized_return = mean_daily_return * 252
    volatility = std_daily_return * np.sqrt(252)
    
    # Sharpe ratio
    if volatility > 0:
        sharpe_ratio = (annualized_return - risk_free_rate) / volatility
    else:
        sharpe_ratio = 0
    
    # Maximum drawdown
    if 'portfolio_value' in history_df.columns:
        cumulative = history_df['portfolio_value']
    else:
        cumulative = (1 + daily_returns).cumprod()
    
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'annualized_return': annualized_return * 100,  # as percentage
        'volatility': volatility * 100,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown * 100
    }


def compare_all_portfolios(copycat_portfolios, custom_portfolios, spy_df, risk_free_rate=None):
    """
    Compare all portfolios and return metrics DataFrame.
    
    Args:
        copycat_portfolios: dict of {name: CopycatPortfolio} or {name: history_df}
        custom_portfolios: dict of {name: CustomPortfolio} or {name: history_df}
        spy_df: SPY benchmark DataFrame
        
    Returns:
        DataFrame with comparison metrics
    """
    if risk_free_rate is None:
        risk_free_rate = get_risk_free_rate()
    
    results = []
    
    # Copycat portfolios
    for name, portfolio in copycat_portfolios.items():
        if hasattr(portfolio, 'history'):
            history = portfolio.history
        else:
            history = portfolio  # Assume it's already a DataFrame
        
        metrics = calculate_portfolio_metrics(history, risk_free_rate)
        metrics['name'] = name
        metrics['type'] = 'Copycat'
        results.append(metrics)
    
    # Custom portfolios
    for name, portfolio in custom_portfolios.items():
        if hasattr(portfolio, 'history'):
            history = portfolio.history
        else:
            history = portfolio
        
        metrics = calculate_portfolio_metrics(history, risk_free_rate)
        metrics['name'] = name
        metrics['type'] = 'Custom'
        results.append(metrics)
    
    # SPY benchmark
    spy_metrics = calculate_portfolio_metrics(spy_df, risk_free_rate)
    spy_metrics['name'] = 'SPY'
    spy_metrics['type'] = 'Benchmark'
    results.append(spy_metrics)
    
    # Create DataFrame
    df = pd.DataFrame(results)
    df = df[['name', 'type', 'annualized_return', 'volatility', 'sharpe_ratio', 'max_drawdown']]
    df.columns = ['Portfolio', 'Type', 'Ann. Return (%)', 'Volatility (%)', 'Sharpe Ratio', 'Max Drawdown (%)']
    
    # Sort by Sharpe ratio
    df = df.sort_values('Sharpe Ratio', ascending=False).reset_index(drop=True)
    
    return df


def print_comparison_table(comparison_df):
    """Print formatted comparison table."""
    print("\n" + "="*90)
    print("PORTFOLIO COMPARISON")
    print("="*90)
    print(f"\nRisk-Free Rate (3M T-Bill): {get_risk_free_rate()*100:.2f}%\n")
    print(comparison_df.to_string(index=False))
    print("\n" + "="*90)

In [122]:
# Compare
custom_dict = {
    'Equal Weight': equal_weight,
    'Equal Value': equal_value,
    'Mean-Variance': mean_var
}

In [123]:
comparison = compare_all_portfolios(copycat_objects, custom_dict, spy_returns)
print(comparison)


C:\Users\Aaron\AppData\Local\Temp\ipykernel_12964\3681617875.py:5: FutureWarning:

YF.download() has changed argument auto_adjust default to True



                   Portfolio       Type  Ann. Return (%)  Volatility (%)  \
0            Earl Blumenauer    Copycat        38.047422       27.588876   
1                Equal Value     Custom        27.570822       21.128382   
2              Mean-Variance     Custom        25.140994       20.872173   
3               Equal Weight     Custom        32.034495       31.552925   
4               Lois Frankel    Copycat        28.995408       30.558622   
5                  Susie Lee    Copycat        63.134558       74.405831   
6            Josh Gottheimer    Copycat        32.245267       36.481104   
7                 Kevin Hern    Copycat       149.633866      206.401022   
8              Pete Sessions    Copycat        20.260651       24.432011   
9             Mikie Sherrill    Copycat        88.451911      126.062524   
10              Nancy Pelosi    Copycat        52.344226       73.993417   
11                       SPY  Benchmark        16.196371       19.889102   
12       Car

## Visualization

In [124]:
def plot_custom_portfolios_comparison(custom_portfolios_dict, spy_df, save_path=None):
    """
    Plot custom portfolios vs SPY.
    Shows both cumulative return (%) and cumulative log return.
    """
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Cumulative Return (%)', 'Cumulative Log Return'),
        vertical_spacing=0.12
    )
    
    colors = {
        'Equal Weight': '#1f77b4',
        'Equal Value': '#ff7f0e', 
        'Mean-Variance': '#2ca02c'
    }
    
    # Top plot: Cumulative Returns
    for name, portfolio in custom_portfolios_dict.items():
        if hasattr(portfolio, 'history'):
            history = portfolio.history
        else:
            history = portfolio
        
        if history is None or history.empty:
            continue
        
        fig.add_trace(go.Scatter(
            x=history['Date'],
            y=history['cumulative_return_pct'],
            mode='lines',
            name=f'{name} (Return %)',
            line=dict(width=2, color=colors.get(name, '#333'))
        ), row=1, col=1)
    
    # SPY cumulative returns
    fig.add_trace(go.Scatter(
        x=spy_df['Date'],
        y=spy_df['cumulative_return_pct'],
        mode='lines',
        name='SPY (Return %)',
        line=dict(width=2, dash='dash', color='gray')
    ), row=1, col=1)
    
    # Bottom plot: Log Returns
    for name, portfolio in custom_portfolios_dict.items():
        if hasattr(portfolio, 'history'):
            history = portfolio.history
        else:
            history = portfolio
        
        if history is None or history.empty or 'cumulative_log_return' not in history.columns:
            continue
        
        fig.add_trace(go.Scatter(
            x=history['Date'],
            y=history['cumulative_log_return'],
            mode='lines',
            name=f'{name} (Log)',
            line=dict(width=2, color=colors.get(name, '#333'))
        ), row=2, col=1)
    
    # SPY log returns
    fig.add_trace(go.Scatter(
        x=spy_df['Date'],
        y=spy_df['cumulative_log_return'],
        mode='lines',
        name='SPY (Log)',
        line=dict(width=2, dash='dash', color='gray')
    ), row=2, col=1)
    
    fig.update_layout(
        title='Custom Portfolios vs SPY',
        template='plotly_white',
        height=700,
        showlegend=True,
        legend=dict(x=0.02, y=0.98),
        hovermode='x unified'
    )
    
    fig.update_xaxes(title_text='Date', row=2, col=1)
    fig.update_yaxes(title_text='Cumulative Return (%)', row=1, col=1)
    fig.update_yaxes(title_text='Cumulative Log Return', row=2, col=1)
    
    if save_path:
        fig.write_html(save_path)
    
    return fig


In [125]:
fig = plot_custom_portfolios_comparison(custom_dict, spy_returns)
fig.show()


In [126]:
test = equal_value.get_portfolio_composition('2020-12-15')
print(test)


Portfolio Composition - 2020-12-15
Weighting: equal_value, Rebalance: quarterly
Total Value:         $218,536.39
Number of Positions: 19

   Asset   Type       Shares       Price         Value  Weight (%)
0    TTD  Stock   183.034307   93.313004  17079.480957        7.82
1     AM  Stock  2787.322844    5.480047  15274.659490        6.99
2   MPLX  Stock   990.917232   14.667245  14534.025727        6.65
3   MIDD  Stock   105.846099  134.350006  14220.424112        6.51
4    PTC  Stock   114.790297  114.050003  13091.833696        5.99
5     ET  Stock  2684.968238    4.695215  12606.502472        5.77
6   QCOM  Stock    90.252066  133.211884  12022.647664        5.50
7   TTWO  Stock    57.471574  194.699997  11189.715237        5.12
8    ATR  Stock    89.911605  121.898293  10960.071079        5.02
9   PYPL  Stock    48.304344  221.088913  10679.554842        4.89
10  AAPL  Stock    84.373013  124.485069  10503.180309        4.81
11     T  Stock   639.722479   16.310692  10434.316215   

In [70]:
test2 = equal_value.get_portfolio_composition('2021-01-05')
print(test2)


Portfolio Composition - 2021-01-05
Weighting: equal_value, Rebalance: quarterly
Total Value:         $217,434.10
Number of Positions: 16

   Asset   Type       Shares       Price         Value  Weight (%)
0     AM  Stock  2634.035280    5.513380  14522.437565        6.68
1    NGL  Stock  5641.272811    2.540000  14328.832724        6.59
2   MPLX  Stock   989.167612   14.452291  14295.738660        6.57
3     ET  Stock  3261.531351    4.339211  14152.471117        6.51
4   PYPL  Stock    57.943443  234.368210  13580.100914        6.25
5   CRWD  Stock    63.917736  211.240005  13501.982880        6.21
6   USAC  Stock  1705.007052    7.876535  13429.547618        6.18
7   AMZN  Stock    83.139985  160.925507  13379.344205        6.15
8   AAPL  Stock   104.817894  127.531975  13367.633020        6.15
9    SPH  Stock  1352.256372    9.857229  13329.501044        6.13
10   TTD  Stock   169.026911   78.853996  13328.447382        6.13
11     V  Stock    64.184232  206.870834  13277.845558   

In [71]:
test3 = equal_value.get_portfolio_composition('2019-05-31')
print(test3)


Portfolio Composition - 2019-05-31
Weighting: equal_value, Rebalance: quarterly
Total Value:         $120,041.67
Number of Positions: 18

   Asset   Type      Shares       Price        Value  Weight (%)
0   TTWO  Stock   69.775703  108.150002  7546.242412        6.29
1    RMD  Stock   67.256103  107.818420  7251.446782        6.04
2    PGR  Stock  111.471925   65.051361  7251.400426        6.04
3    ATR  Stock   67.578750  104.075577  7033.297364        5.86
4     LH  Stock   52.447398  133.452560  6999.239488        5.83
5    JPM  Stock   78.865070   88.063644  6945.145447        5.79
6   TDOC  Stock  118.430458   58.119999  6883.178116        5.73
7    CDW  Stock   74.072823   91.058052  6744.926976        5.62
8   DXCM  Stock  221.149736   30.325001  6706.365914        5.59
9    TTD  Stock  332.646289   19.881001  6613.341034        5.51
10     T  Stock  456.010360   14.380519  6557.665606        5.46
11   TXN  Stock   75.642763   86.170883  6518.203664        5.43
12   ITW  Stock 

In [72]:
test4 = mean_var.get_portfolio_composition('2021-01-5')
print(test4)


Portfolio Composition - 2021-01-05
Weighting: mean_variance, Rebalance: quarterly
Total Value:         $195,690.69
Number of Positions: 16

   Asset   Type       Shares       Price         Value  Weight (%)
0     AM  Stock  2370.631752    5.513380  13070.193809        6.68
1    NGL  Stock  5077.145530    2.540000  12895.949452        6.59
2   MPLX  Stock   890.250851   14.452291  12866.164794        6.57
3     ET  Stock  2935.378216    4.339211  12737.224006        6.51
4   PYPL  Stock    52.149098  234.368210  12222.090822        6.25
5   CRWD  Stock    57.525962  211.240005  12151.784592        6.21
6   USAC  Stock  1534.506347    7.876535  12086.592857        6.18
7   AMZN  Stock    74.825987  160.925507  12041.409785        6.15
8   AAPL  Stock    94.336105  127.531975  12030.869718        6.15
9    SPH  Stock  1217.030735    9.857229  11996.550939        6.13
10   TTD  Stock   152.124220   78.853996  11995.602644        6.13
11     V  Stock    57.765809  206.870834  11950.061002 

In [74]:
custom_dict['Equal Value'].history

,Date,stock_value,n_holdings,daily_return,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,100000.000000,18,0.000000,0.000000,0.000000,0.000000,0.000000
1,2019-01-04,104042.825980,18,0.040428,0.040428,4.042826,0.039632,0.039632
2,2019-01-07,106630.208569,18,0.024868,0.066302,6.630209,0.024564,0.064197
3,2019-01-08,107927.160170,18,0.012163,0.079272,7.927160,0.012090,0.076286
4,2019-01-09,109156.637550,18,0.011392,0.091566,9.156638,0.011327,0.087614
...,...,...,...,...,...,...,...,...
1561,2024-12-27,555103.439712,14,-0.010691,4.551034,455.103440,-0.010749,1.713984
1562,2024-12-30,552386.864573,14,-0.004894,4.523869,452.386865,-0.004906,1.709078
1563,2024-12-31,552386.864573,14,0.000000,4.523869,452.386865,0.000000,1.709078
1564,2025-01-01,552386.864573,14,0.000000,4.523869,452.386865,0.000000,1.709078


In [76]:
custom_dict['Equal Weight'].history

,Date,stock_value,n_holdings,daily_return,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,100000.000000,18,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
1,2019-01-04,104517.810008,18,4.517810e-02,0.045178,4.517810,4.418730e-02,0.044187
2,2019-01-07,107882.915349,18,3.219648e-02,0.078829,7.882915,3.168903e-02,0.075876
3,2019-01-08,109324.775763,18,1.336505e-02,0.093248,9.324776,1.327652e-02,0.089153
4,2019-01-09,110449.378936,18,1.028681e-02,0.104494,10.449379,1.023426e-02,0.099387
...,...,...,...,...,...,...,...,...
1561,2024-12-27,711259.199418,14,4.878697e-04,6.112592,611.259199,4.877508e-04,1.961867
1562,2024-12-30,716109.835299,14,6.819786e-03,6.161098,616.109835,6.796637e-03,1.968663
1563,2024-12-31,716109.835299,14,2.220446e-16,6.161098,616.109835,2.220446e-16,1.968663
1564,2025-01-01,716109.835299,14,0.000000e+00,6.161098,616.109835,0.000000e+00,1.968663


In [96]:
custom_dict['Mean-Variance'].history

,Date,stock_value,n_holdings,daily_return,cumulative_return,cumulative_return_pct,log_return,cumulative_log_return
0,2019-01-03,90000.000000,18,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
1,2019-01-04,93638.543382,18,4.042826e-02,0.040428,4.042826,3.963242e-02,0.039632
2,2019-01-07,95967.187712,18,2.486844e-02,0.066302,6.630209,2.456425e-02,0.064197
3,2019-01-08,97134.444153,18,1.216308e-02,0.079272,7.927160,1.208970e-02,0.076286
4,2019-01-09,98240.973795,18,1.139173e-02,0.091566,9.156638,1.132734e-02,0.087614
...,...,...,...,...,...,...,...,...
1561,2024-12-27,499593.095741,14,-1.069121e-02,4.551034,455.103440,-1.074877e-02,1.713984
1562,2024-12-30,497148.178115,14,-4.893818e-03,4.523869,452.386865,-4.905832e-03,1.709078
1563,2024-12-31,497148.178115,14,6.661338e-16,4.523869,452.386865,6.661338e-16,1.709078
1564,2025-01-01,497148.178115,14,0.000000e+00,4.523869,452.386865,0.000000e+00,1.709078
